In [15]:
import primer3
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd
import itertools
import re
from scipy.spatial import distance
import json
from Bio.SeqUtils import MeltingTemp as mt
import random
import pprint
import string

In [16]:
#Import primers
orthogonal_F = pd.read_excel('RR_orthogonalFv1_plate.xlsx')
orthogonal_R = pd.read_excel('RR_orthogonalRv1_plate.xlsx')

In [17]:
#Isolate 20-nt variable primer binding region and only use that to align to sequences
orthogonal_F['PrimerEnd'] = orthogonal_F.Sequence.apply(lambda x: x[-20:])
orthogonal_R['PrimerEnd'] = orthogonal_R.Sequence.apply(lambda x: x[-20:])

In [18]:
# Exclude poor primers determined empirically
bad_primers_F = ["A9", "C3", "D3", "E3", "G4", "G6", "H4"]
bad_primers_R = ["A9", "C6", "D3", "D7", "E10", "E3", "E6", "H4"]
red_primers = ["A8","A9","C3","C4","C6","C8","C9","H2","D3","D4","D8","D9","E3","E6","F3","F4","G3","G4","G6","H3","H4","H5","H7"]

orthogonal_F['Exclude'] = orthogonal_F['Well Position'].map(lambda x: True if x in bad_primers_F else False)
orthogonal_R['Exclude'] = orthogonal_R['Well Position'].map(lambda x: True if x in bad_primers_R else False)

orthogonal_F['Worse'] = orthogonal_F['Well Position'].map(lambda x: True if x in red_primers else False)
orthogonal_R['Worse'] = orthogonal_R['Well Position'].map(lambda x: True if x in red_primers else False)

In [19]:
#Check for BsaI sites and remove any primers with them
bsaI_seq = Seq('GGTCTC')
orthogonal_F['BsaI_Site_Present'] = orthogonal_F.Sequence.apply(lambda x: ( (str(bsaI_seq) in x) | (str(bsaI_seq.reverse_complement()) in x) ))
orthogonal_R['BsaI_Site_Present'] = orthogonal_R.Sequence.apply(lambda x: ( (str(bsaI_seq) in x) | (str(bsaI_seq.reverse_complement()) in x) ))


In [25]:
# Adapted from Willow Coyote-Maestas' DIMPLE paper, cite doi: 10.1186/s13059-023-02880-6
def check_nonspecific(primer, fragment, Tm_verb = 20, Tm_rem = 28, primer3_shift = 4, verbose=True):
    non = []
    # Forward
    for i in range(len(fragment) - len(primer)):  # Scan each position
        match = [
            primer[j].lower() == fragment[i + j].lower() for j in range(len(primer))
        ]
        first = 10
        for k in range(len(match) - 3):
            if (match[k] and match[k + 1] and match[k + 3]) or (
                match[k] and match[k + 1] and match[k + 2]
            ):
                first = k
                break
        if (
            sum(match[first:]) > len(primer[first:]) * 0.8
            and sum(match[first:]) > 6
            and match[-1]
        ):  # string compare - sum of matched nt is greater than 80%
            try:
                melt = mt.Tm_NN(
                    primer[first:],
                    c_seq=fragment[i + first : i + len(primer)].complement(),
                    nn_table=mt.DNA_NN2,
                    de_table=mt.DNA_DE1,
                    imm_table=mt.DNA_IMM1,
                )
                if verbose == True:
                    if melt > Tm_verb:
                        print("Found non-specific match at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(round(melt, 1)))
                if melt > Tm_rem:
                    non.append(True)
            except ValueError as valerr:
                # use primer3 instead, as mt.DNA_NN2 table does not have enough information to compute Tm
                result = primer3.calcHeterodimer(str(primer[first:]), 
                                                 str(fragment[i + first : i + len(primer)].complement())
                                                )
                melt = result.tm
                if verbose == True:
                    if melt > Tm_verb-primer3_shift:
                        print("Found non-specific match using Primer3 at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(round(melt, 1)))
                if melt > Tm_rem-primer3_shift:
                    non.append(True)
                    
    # Reverse
    fragment = fragment.reverse_complement()
    for i in range(len(fragment) - len(primer)):
        match = [
            primer[j].lower() == fragment[i + j].lower() for j in range(len(primer))
        ]
        first = 10
        for k in range(0, len(match) - 3, 1):
            if match[k] and match[k + 1] and match[k + 3]:
                first = k
                break
        if (
            sum(match[first:]) > len(primer[first:]) * 0.8
            and sum(match[first:]) > 6
            and match[-1]
        ):  # string compare - sum of matched nt is greater than 80%
            try:
                melt = mt.Tm_NN(
                    primer[first:],
                    c_seq=fragment[i + first : i + len(primer)].complement(),
                    nn_table=mt.DNA_NN2,
                    de_table=mt.DNA_DE1,
                    imm_table=mt.DNA_IMM1,
                )
                if verbose == True:
                    if melt > Tm_verb:
                        print("Found non-specific match at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(melt))
                if melt > Tm_rem:
                    non.append(True)
            except ValueError as valerr:
                # use primer3 instead, as mt.DNA_NN2 table does not have enough information to compute Tm
                result = primer3.calcHeterodimer(str(primer[first:]), 
                                                 str(fragment[i + first : i + len(primer)].complement())
                                                )
                melt = result.tm
                if verbose == True:
                    if melt > Tm_verb-primer3_shift:
                        print("Found non-specific match using Primer3 at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(round(melt, 1)))
                if melt > Tm_rem-primer3_shift:
                    non.append(True)
    return sum(non)


In [26]:
# Use refseq MANE annotations to find gene cDNA sequences
refseq_MANE_summary_filepath = 'MANE.GRCh38.v1.3.summary.txt'
refseq_MANE_filepath = "MANE.GRCh38.v1.3.refseq_rna.gbff"
refseq_MANE_summary = pd.read_csv(refseq_MANE_summary_filepath, sep='\t')
refseq_MANE_stream = SeqIO.parse(refseq_MANE_filepath, "genbank")

# genes for assay
genes_of_interest = \
['VWF']

# find correct transcripts
genes_to_MANE_transcripts = refseq_MANE_summary.\
    query('MANE_status == "MANE Select"').\
    query('symbol in @genes_of_interest')

# find refseq
gene_transcript_dict = {}
for rec in refseq_MANE_stream:
    if rec.id in genes_to_MANE_transcripts['RefSeq_nuc'].values:
        print('Processing ' + rec.description)
        for feature in rec.features:
            if feature.type == "CDS":
                gene_transcript_dict[rec.id] = [rec.description,
                                                str(feature.location.extract(rec).seq)]
genes_to_MANE_transcripts = genes_to_MANE_transcripts.\
    merge(pd.DataFrame.from_dict(gene_transcript_dict, orient='index', columns=['Description', 'CDS_Seq']),
             left_on='RefSeq_nuc', right_index=True)


Processing Homo sapiens von Willebrand factor (VWF), mRNA


In [27]:
for gene in genes_of_interest:
    print(gene)
    gene_cds_seq = Seq(genes_to_MANE_transcripts[genes_to_MANE_transcripts['symbol']==gene]['CDS_Seq'].values[0].upper())
    print('CDS: ' + str(gene_cds_seq))
    print('AA: ' + str(gene_cds_seq.translate()))


VWF
CDS: ATGATTCCTGCCAGATTTGCCGGGGTGCTGCTTGCTCTGGCCCTCATTTTGCCAGGGACCCTTTGTGCAGAAGGAACTCGCGGCAGGTCATCCACGGCCCGATGCAGCCTTTTCGGAAGTGACTTCGTCAACACCTTTGATGGGAGCATGTACAGCTTTGCGGGATACTGCAGTTACCTCCTGGCAGGGGGCTGCCAGAAACGCTCCTTCTCGATTATTGGGGACTTCCAGAATGGCAAGAGAGTGAGCCTCTCCGTGTATCTTGGGGAATTTTTTGACATCCATTTGTTTGTCAATGGTACCGTGACACAGGGGGACCAAAGAGTCTCCATGCCCTATGCCTCCAAAGGGCTGTATCTAGAAACTGAGGCTGGGTACTACAAGCTGTCCGGTGAGGCCTATGGCTTTGTGGCCAGGATCGATGGCAGCGGCAACTTTCAAGTCCTGCTGTCAGACAGATACTTCAACAAGACCTGCGGGCTGTGTGGCAACTTTAACATCTTTGCTGAAGATGACTTTATGACCCAAGAAGGGACCTTGACCTCGGACCCTTATGACTTTGCCAACTCATGGGCTCTGAGCAGTGGAGAACAGTGGTGTGAACGGGCATCTCCTCCCAGCAGCTCATGCAACATCTCCTCTGGGGAAATGCAGAAGGGCCTGTGGGAGCAGTGCCAGCTTCTGAAGAGCACCTCGGTGTTTGCCCGCTGCCACCCTCTGGTGGACCCCGAGCCTTTTGTGGCCCTGTGTGAGAAGACTTTGTGTGAGTGTGCTGGGGGGCTGGAGTGCGCCTGCCCTGCCCTCCTGGAGTACGCCCGGACCTGTGCCCAGGAGGGAATGGTGCTGTACGGCTGGACCGACCACAGCGCGTGCAGCCCAGTGTGCCCTGCTGGTATGGAGTATAGGCAGTGTGTGTCCCCTTGCGCCAGGACCTGCCAGAGCCTGCACATCAATGAAATGTGTCAGGAGCGATGCGTGGATGGCTGCAGCT

In [41]:
# Define genes
# clotting proteins:

# truncated F8 (block1_notag) (2_1)
FRED_SB1 = Seq('atgcaaatagagctctctacctgcttctttctgtgccttttgcgattctgctttagtgccaccagaagatactacctgggtgcagtggaactgtcatgggactatatgcaaagtgatctcggtgagctgcctgtggacgcaagatttcctcctagagtgccaaaatcttttccattcaacacctcagtcgtgtacaaaaagactctgtttgtagaattcacggatcaccttttcaacatcgctaagccaaggccaccctggatgggtctgctaggtcctaccatccaggctgaggtttatgatacagtggtcattacacttaagaacatggcttcccatcctgtcagtcttcatgctgttggtgtatcctactggaaagcttctgagggagctgaatatgatgatcagaccagtcaaagggagaaagaagatgataaagtcttccctggtggaagccatacatatgtctggcaggtcctgaaagagaatggtccaatggcctctgacccactgtgccttacctactcatatctttctcatgtggacctggtaaaagacttgaattcaggcctcattggagccctactagtatgtagagaagggagtctggccaaggaaaagacacagaccttgcacaaatttatactactttttgctgtatttgatgaagggaaaagttggcactcagaaacaaagaactccttgatgcaggatagggatgctgcatctgctcgggcctggcctaaaatgcacacagtcaatggttatgtaaacagatctctgccaggtctgattggatgccacaggaaatcagtctattggcatgtgattggaatgggcaccactcctgaagtgcactcaatattcctcgaaggtcacacatttcttgtgaggaaccatcgccaggcgtccttggaaatctcgccaataactttccttactgctcaaacactcttgatggaccttggacagtttctactgttttgtcatatctcttcccaccaacatgatggcatggaagcttatgtcaaagtagacagctgtccagaggaaccccaactacgaatgaaaaataatgaagaagcggaagactatgatgatgatcttactgattctgaaatggatgtggtcaggtttgatgatgacaactctccttcctttatccaaattcgctcagttgccaagaagcatcctaaaacttgggtacattacattgctgctgaagaggaggactgggactatgctcccttagtcctcgcccccgatgacagaagttataaaagtcaatatttgaacaatggccctcagcggattggtaggaagtacaaaaaagtccgatttatggcatacacagatgaaacctttaagactcgtgaagctattcagcatgaatcaggaatcttgggacctttactttatggggaagttggagacacactgttgattatatttaagaatcaagcaagcagaccatataacatctaccctcacggaatcactgatgtccgtcctttgtattcaaggagattaccaaaaggtgtaaaacatttgaaggattttccaattctgccaggagaaatattcaaatataaatggacagtgactgtagaagatgggccaactaaatcagatcctcggtgcctgacccgctattactct'.upper())

# truncated F8 (block2_notag) (1_2)
FRED_SB2 = Seq('ttcaaatataaatggacagtgactgtagaagatgggccaactaaatcagatcctcggtgcctgacccgctattactctagtttcgttaatatggagagagatctagcttcaggactcattggccctctcctcatctgctacaaagaatctgtagatcaaagaggaaaccagataatgtcagacaagaggaatgtcatcctgttttctgtatttgatgagaaccgaagctggtacctcacagagaatatacaacgctttctccccaatccagctggagtgcagcttgaggatccagagttccaagcctccaacatcatgcacagcatcaatggctatgtttttgatagtttgcagttgtcagtttgtttgcatgaggtggcatactggtacattctaagcattggagcacagactgacttcctttctgtcttcttctctggatataccttcaaacacaaaatggtctatgaagacacactcaccctattcccattctcaggagaaactgtcttcatgtcgatggaaaacccaggtctatggattctggggtgccacaactcagactttcggaacagaggcatgaccgccttactgaaggtttctagttgtgacaagaacactggtgattattacgaggacagttatgaagatatttcagcatacttgctgagtaaaaacaatgccattgaaccaagaagcttctcccagaattcaagacaccctagcactaggcaaaagcaatttaatgccaccacaattccagaaaatgacatagagaagactgacccttggtttgcacacagaacacctatgcctaaaatacaaaatgtctcctctagtgatttgttgatgctcttgcgacagagtcctactccacatgggctatccttatctgatctccaagaagccaaatatgagactttttctgatgatccatcacctggagcaatagacagtaataacagcctgtctgaaatgacacacttcaggccacagctccatcacagtggggacatggtatttacccctgagtcaggcctccaattaagattaaatgagaaactggggacaactgcagcaacagagttgaagaaacttgatttcaaagtttctagtacatcaaataatctgatttcaacaattccatcagacaatttggcagcaggtactgataatacaagttccttaggacccccaagtatgccagttcattatgatagtcaattagataccactctatttggcaaaaagtcatctccccttactgagtctggtggacctctgagcttgagtgaagaaaataatgattcaaagttgttagaatcaggtttaatgaatagccaagaaagttcatggggaaaaaatgtatcgtcacgggaaataactcgtactactcttcagtcagatcaagaggaaattgactatgatgataccatatcagttgaaatgaagaaggaagattttgacatttatgatgaggatgaaaatcagagcccccgcagctttcaaaagaaaacacgacactattttattgctgcagtggagaggctctgggattatgggatgagtagctccccacatgttctaagaaacagggctcagagtggcagtgtccctcagttcaagaaagttgttttccaggaatttactgatggctcctttactcagcccttataccgtggagaactaaatgaacatttgggactcctggggccatatataaga'.upper())

# truncated F8 (block3_ctag) (3_2)
FRED_SB3 = Seq('ccatatataagagcagaagttgaagataatatcatggtaactttcagaaatcaggcctctcgtccctattccttctattctagccttatttcttatgaggaagatcagaggcaaggagcagaacctagaaaaaactttgtcaagcctaatgaaaccaaaacttacttttggaaagtgcaacatcatatggcacccactaaagatgagtttgactgcaaagcctgggcttatttctctgatgttgacctggaaaaagatgtgcactcaggcctgattggaccccttctggtctgccacactaacacactgaaccctgctcatgggagacaagtgacagtacaggaatttgctctgtttttcaccatctttgatgaaaccaaaagctggtacttcactgaaaatatggaaagaaactgcagggctccctgcaatatccagatggaagatcccacttttaaagagaattatcgcttccatgcaatcaatggctacataatggatacactacctggcttagtaatggctcaggatcaaaggattcgatggtatctgctcagcatgggcagcaatgaaaacatccattctattcatttcagtggacatgtgttcactgtacgaaaaaaagaggagtataaaatggcactgtacaatctctatccaggtgtttttgagacagtggaaatgttaccatccaaagctggaatttggcgggtggaatgccttattggcgagcatctacatgctgggatgagcacactttttctggtgtacagcaataagtgtcagactcccctgggaatggcttctggacacattagagattttcagattacagcttcaggacaatatggacagtgggccccaaagctggccagacttcattattccggatcaatcaatgcctggagcaccaaggagcccttttcttggatcaaggtggatctgttggcaccaatgattattcacggcatcaagacccagggtgcccgtcagaagttctccagcctctacatctctcagtttatcatcatgtatagtcttgatgggaagaagtggcagacttatcgaggaaattccactggaaccttaatggtcttctttggcaatgtggattcatctgggataaaacacaatatttttaaccctccaattattgctcgatacatccgtttgcacccaactcattatagcattcgcagcactcttcgcatggagttgatgggctgtgatttaaatagttgcagcatgccattgggaatggagagtaaagcaatatcagatgcacagattactgcttcatcctactttaccaatatgtttgccacctggtcaccttcaaaagctcgacttcacctccaagggaggagtaatgcctggaggcctcaggtgaataatccaaaagagtggctgcaagtggacttccagaagacaatgaaagtcacaggagtaactactcagggagtaaaatctctgcttaccagcatgtatgtgaaggagttcctcatctccagcagtcaagatggccatcagtggactctcttttttcagaatggcaaagtaaaggtttttcagggaaatcaagactccttcacacctgtggtgaactctctagacccaccgttactgactcgctaccttcgaattcacccccagagttgggtgcaccagattgccctgaggatggaggttctgggctgcgaggcacaggacctctac'.upper())

# VWF (block1_notag) (3_1)
VWF_SB1 = Seq('ATGATTCCTGCCAGATTTGCCGGGGTGCTGCTTGCTCTGGCCCTCATTTTGCCAGGGACCCTTTGTGCAGAAGGAACTCGCGGCAGGTCATCCACGGCCCGATGCAGCCTTTTCGGAAGTGACTTCGTCAACACCTTTGATGGGAGCATGTACAGCTTTGCGGGATACTGCAGTTACCTCCTGGCAGGGGGCTGCCAGAAACGCTCCTTCTCGATTATTGGGGACTTCCAGAATGGCAAGAGAGTGAGCCTCTCCGTGTATCTTGGGGAATTTTTTGACATCCATTTGTTTGTCAATGGTACCGTGACACAGGGGGACCAAAGAGTCTCCATGCCCTATGCCTCCAAAGGGCTGTATCTAGAAACTGAGGCTGGGTACTACAAGCTGTCCGGTGAGGCCTATGGCTTTGTGGCCAGGATCGATGGCAGCGGCAACTTTCAAGTCCTGCTGTCAGACAGATACTTCAACAAGACCTGCGGGCTGTGTGGCAACTTTAACATCTTTGCTGAAGATGACTTTATGACCCAAGAAGGGACCTTGACCTCGGACCCTTATGACTTTGCCAACTCATGGGCTCTGAGCAGTGGAGAACAGTGGTGTGAACGGGCATCTCCTCCCAGCAGCTCATGCAACATCTCCTCTGGGGAAATGCAGAAGGGCCTGTGGGAGCAGTGCCAGCTTCTGAAGAGCACCTCGGTGTTTGCCCGCTGCCACCCTCTGGTGGACCCCGAGCCTTTTGTGGCCCTGTGTGAGAAGACTTTGTGTGAGTGTGCTGGGGGGCTGGAGTGCGCCTGCCCTGCCCTCCTGGAGTACGCCCGGACCTGTGCCCAGGAGGGAATGGTGCTGTACGGCTGGACCGACCACAGCGCGTGCAGCCCAGTGTGCCCTGCTGGTATGGAGTATAGGCAGTGTGTGTCCCCTTGCGCCAGGACCTGCCAGAGCCTGCACATCAATGAAATGTGTCAGGAGCGATGCGTGGATGGCTGCAGCTGCCCTGAGGGACAGCTCCTGGATGAAGGCCTCTGCGTGGAGAGCACCGAGTGTCCCTGCGTGCATTCCGGAAAGCGCTACCCTCCCGGCACCTCCCTCTCTCGAGACTGCAACACaTGCATTTGCCGAAACAGCCAGTGGATCTGCAGCAATGAAGAATGTCCAGGGGAGTGCCTTGTCACAGGTCAATCACACTTCAAGAGCTTTGACAACAGATACTTCACCTTCAGTGGGATCTGCCAGTACCTGCTGGCCCGGGATTGCCAGGACCACTCCTTCTCCATTGTCATTGAGACTGTCCAGTGTGCTGATGACCGCGACGCTGTGTGCACCCGCTCCGTCACCGTCCGGCTGCCTGGCCTGCACAACAGCCTTGTGAAACTGAAGCATGGGGCAGGAGTTGCCATGGATGGCCAGGACGTCCAGCTCCCCCTCCTGAAAGGTGACCTCCGCATCCAGCATACAGTGACGGCCTCCGTGCGCCTCAGCTACGGGGAGGACCTGCAGATGGACTGGGATGGCCGCGGGAGGCTGCTGGTGAAGCTGTCCCCCGTCTATGCCGGGAAGACCTGCGGCCTGTGTGGGAATTACAATGGCAACCAGGGCGACGACTTCCTTACCCCCTCTGGGCTGGCGGAGCCCCGGGTGGAGGACTTCGGGAACGCCTGGAAGCTGCACGGGGACTGCCAGGACCTG'.upper())

# VWF (block2_notag) (2_2)
VWF_SB2 = Seq('CAGGACCTGCAGAAGCAGCACAGCGATCCCTGCGCCCTCAACCCGCGCATGACCAGGTTCTCCGAGGAGGCGTGCGCGGTCCTGACGTCCCCCACATTCGAGGCCTGCCATCGTGCCGTCAGCCCGCTGCCCTACCTGCGGAACTGCCGCTACGACGTGTGCTCCTGCTCGGACGGCCGCGAGTGCCTGTGCGGCGCCCTGGCCAGCTATGCCGCGGCCTGCGCGGGGAGAGGCGTGCGCGTCGCGTGGCGCGAGCCAGGCCGCTGTGAGCTGAACTGCCCGAAAGGCCAGGTGTACCTGCAGTGCGGGACCCCCTGCAACCTGACCTGCCGCTCTCTCTCTTACCCGGATGAGGAATGCAATGAGGCCTGCCTGGAGGGCTGCTTCTGCCCCCCAGGGCTCTACATGGATGAGAGGGGGGACTGCGTGCCCAAGGCCCAGTGCCCCTGTTACTATGACGGTGAGATCTTCCAGCCAGAAGACATCTTCTCAGACCATCACACCATGTGCTACTGTGAGGATGGCTTCATGCACTGTACCATGAGTGGAGTCCCCGGAAGCTTGCTGCCTGACGCTGTCCTCAGCAGTCCCCTGTCTCATCGCAGCAAAAGGAGCCTATCCTGTCGGCCCCCCATGGTCAAGCTGGTGTGTCCCGCTGACAACCTGCGGGCTGAAGGGCTCGAGTGTACCAAAACGTGCCAGAACTATGACCTGGAGTGCATGAGCATGGGCTGTGTCTCTGGCTGCCTCTGCCCCCCGGGCATGGTCCGGCATGAGAACAGATGTGTGGCCCTGGAAAGGTGTCCCTGCTTCCATCAGGGCAAGGAGTATGCCCCTGGAGAAACAGTGAAGATTGGCTGCAACACTTGTGTCTGTCAGGACCGGAAGTGGAACTGCACAGACCATGTGTGTGATGCCACGTGCTCCACGATCGGCATGGCCCACTACCTCACCTTCGACGGGCTCAAATACCTGTTCCCCGGGGAGTGCCAGTACGTTCTGGTGCAGGATTACTGCGGCAGTAACCCTGGGACCTTTCGGATCCTAGTGGGGAATAAGGGATGCAGCCACCCCTCAGTGAAATGCAAGAAACGGGTCACCATCCTGGTGGAGGGAGGAGAGATTGAGCTGTTTGACGGGGAGGTGAATGTGAAGAGGCCCATGAAGGATGAGACTCACTTTGAGGTGGTGGAGTCTGGCCGGTACATCATTCTGCTGCTGGGCAAAGCCCTCTCCGTGGTCTGGGACCGCCACCTGAGCATCTCCGTGGTCCTGAAGCAGACATACCAGGAGAAAGTGTGTGGCCTGTGTGGGAATTTTGATGGCATCCAGAACAATGACCTCACCAGCAGCAACCTCCAAGTGGAGGAAGACCCTGTGGACTTTGGGAACTCCTGGAAAGTGAGCTCGCAGTGTGCTGACACCAGAAAAGTGCCTCTGGACTCATCCCCTGCCACaTGCCATAACAACATCATGAAGCAGACGATGGTGGATTCCTCCTGTAGAATCCTTACCAGTGACGTCTTCCAGGACTGCAACAAGCTGGTGGACCCCGAGCCATATCTGGATGTCTGCATTTACGACACaTGCTCCTGTGAGTCCATTGGGGACTGCGCCTGCTTCTGCGACACCATTGCTGCCTATGCCCACGTGTGTGCCCAGCATGGCAAGGTGGTGACCTGGAGGACGGCCACATTGTGCCCCCAGAGCTGCGAGGAGAGGAATCTCCGG'.upper())

# VWF (block3_notag) (5_3)
VWF_SB3 = Seq('CGGGAGAACGGGTATGAGTGTGAGTGGCGCTATAACAGCTGTGCACCcGCCTGTCAAGTCACGTGTCAGCACCCTGAGCCACTGGCCTGCCCTGTGCAGTGTGTGGAGGGCTGCCATGCCCACTGCCCTCCAGGGAAAATCCTGGATGAGCTTTTGCAGACCTGCGTTGACCCTGAAGACTGTCCAGTGTGTGAGGTGGCTGGCCGGCGTTTTGCCTCAGGAAAGAAAGTCACCTTGAATCCCAGTGACCCTGAGCACTGCCAGATTTGCCACTGTGATGTTGTCAACCTCACCTGTGAAGCCTGCCAGGAGCCGGGAGGCCTGGTGGTGCCTCCCACAGATGCCCCGGTGAGCCCCACCACTCTGTATGTGGAGGACATCTCGGAACCGCCGTTGCACGATTTCTACTGCAGCAGGCTACTGGACCTGGTCTTCCTGCTGGATGGCTCCTCCAGGCTGTCCGAGGCTGAGTTTGAAGTGCTGAAGGCCTTTGTGGTGGACATGATGGAGCGGCTGCGCATCTCCCAGAAGTGGGTCCGCGTGGCCGTGGTGGAGTACCACGACGGCTCCCACGCCTACATCGGGCTCAAGGACCGGAAGCGACCGTCAGAGCTGCGGCGCATTGCCAGCCAGGTGAAGTATGCGGGCAGCCAGGTGGCCTCCACCAGCGAGGTCTTGAAATACACACTGTTCCAAATCTTCAGCAAGATCGACCGCCCTGAAGCCTCCCGCATCACCCTGCTCCTGATGGCCAGCCAGGAGCCCCAACGGATGTCCCGGAACTTTGTCCGCTACGTCCAGGGCCTGAAGAAGAAGAAGGTCATTGTGATCCCGGTGGGCATTGGGCCCCATGCCAACCTCAAGCAGATCCGCCTCATCGAGAAGCAGGCCCCTGAGAACAAGGCCTTCGTGCTGAGCAGTGTGGATGAGCTGGAGCAGCAAAGGGACGAGATCGTTAGCTACCTCTGTGACCTTGCCCCTGAAGCCCCTCCTCCTACTCTGCCCCCCGACATGGCACAAGTCACTGTGGGCCCGGGGCTCTTGGGGGTTTCGACCCTGGGGCCCAAGAGGAACTCCATGGTTCTGGATGTGGCGTTCGTCCTGGAAGGATCGGACAAAATTGGTGAAGCCGACTTCAACAGGAGCAAGGAGTTCATGGAGGAGGTGATTCAGCGGATGGATGTGGGCCAGGACAGCATCCACGTCACGGTGCTGCAGTACTCCTACATGGTGACTGTGGAGTACCCCTTCAGCGAGGCACAGTCCAAAGGGGACATCCTGCAGCGGGTGCGAGAGATCCGCTACCAGGGCGGCAACAGGACCAACACTGGGCTGGCCCTGCGGTACCTCTCTGACCACAGCTTCTTGGTCAGCCAGGGTGACCGGGAGCAGGCGCCCAACCTGGTCTACATGGTCACCGGAAATCCTGCCTCTGATGAGATCAAGAGGCTGCCTGGAGACATCCAGGTGGTGCCCATTGGAGTGGGCCCTAATGCCAACGTGCAGGAGCTGGAGAGGATTGGCTGGCCCAATGCCCCTATCCTCATCCAGGACTTTGAAACGCTCCCCCGAGAGGCTCCTGACCTGGTGCTGCAGAGGTGCTGCTCCGGAGAGGGGCTGCAGATCCCCACCCTCTCCCCTGCACCTGACTGCAGCCAGCCCCTGGACGTGATCCTTCTCCTGGATGGCTCCTCC'.upper())

# VWF (block4_notag) (3_4)
VWF_SB4 = Seq('GTGATCCTTCTCCTGGATGGCTCCTCCAGTTTCCCAGCTTCTTATTTTGATGAAATGAAGAGTTTCGCCAAGGCTTTCATTTCAAAAGCCAATATAGGGCCTAGACTCACTCAGGTGTCAGTGCTGCAGTATGGAAGCATCACCACCATTGACGTGCCATGGAACGTGGTCCCGGAGAAAGCCCATTTGCTGAGCCTTGTGGACGTCATGCAGCGGGAGGGAGGCCCCAGCCAAATCGGGGATGCCTTGGGCTTTGCTGTGCGATACTTGACTTCAGAAATGCATGGTGCCAGGCCGGGAGCCTCAAAGGCGGTGGTCATCCTGGTCACGGACGTGTCTGTGGATTCAGTGGATGCAGCAGCTGATGCCGCCAGGTCCAACAGAGTGACAGTGTTCCCTATTGGAATTGGAGATCGCTACGATGCAGCCCAGCTACGGATCTTGGCAGGCCCAGCAGGCGACTCCAACGTGGTGAAGCTCCAGCGAATCGAAGACCTCCCTACCATGGTCACCTTGGGCAATTCCTTCCTCCACAAACTGTGCTCTGGATTTGTTAGGATTTGCATGGATGAGGATGGGAATGAGAAGAGGCCCGGGGACGTCTGGACCTTGCCAGACCAGTGCCACACCGTGACTTGCCAGCCAGATGGCCAGACCTTGCTGAAGAGTCATCGGGTCAACTGTGACCGGGGGCTGAGGCCTTCGTGCCCTAACAGCCAGTCCCCTGTTAAAGTGGAAGAaACCTGTGGCTGCCGCTGGACCTGCCCCTGCGTGTGCACAGGCAGCTCCACTCGGCACATCGTGACCTTTGATGGGCAGAATTTCAAGCTGACTGGCAGCTGTTCTTATGTCCTATTTCAAAACAAGGAGCAGGACCTGGAGGTGATTCTCCATAATGGTGCCTGCAGCCCTGGAGCAAGGCAGGGCTGCATGAAATCCATCGAGGTGAAGCACAGTGCCCTCTCCGTCGAGCTGCACAGTGACATGGAGGTGACGGTGAATGGGAGACTGGTgTCTGTTCCTTACGTGGGTGGGAACATGGAAGTCAACGTTTATGGTGCCATCATGCATGAGGTCAGATTCAATCACCTTGGTCACATCTTCACATTCACTCCACAAAACAATGAGTTCCAACTGCAGCTCAGCCCCAAGACTTTTGCTTCAAAGACGTATGGTCTGTGTGGGATCTGTGATGAGAACGGAGCCAATGACTTCATGCTGAGGGATGGCACAGTCACCACAGACTGGAAAACACTTGTTCAGGAATGGACTGTGCAGCGGCCAGGGCAGACGTGCCAGCCCATCCTGGAGGAGCAGTGTCTTGTCCCCGACAGCTCCCACTGCCAGGTCCTCCTCTTACCACTGTTTGCTGAATGCCACAAGGTCCTGGCTCCAGCCACATTCTATGCCATCTGCCAGCAGGACAGTTGCCACCAGGAGCAAGTGTGTGAGGTGATCGCCTCTTATGCCCACCTCTGTCGGACCAACGGGGTCTGCGTTGACTGGAGGACACCTGATTTCTGTGCTATGTCATGCCCACCATCTCTGGTCTACAACCACTGTGAGCATGGCTGTCCCCGGCACTGTGATGGCAACGTGAGCTCCTGTGGGGACCATCCCTCCGAAGGCTGTTTCTGCCCTCCAGATAAAGTCATGTTGGAAGGCAGCTGTGTCCCTGAAGAGGCCTGC'.upper())

# VWF (block5_ctag) (2_5)
VWF_SB5 = Seq('GAGGCCTGCACTCAGTGCATTGGTGAGGATGGAGTCCAGCACCAGTTCCTGGAAGCCTGGGTCCCGGACCACCAGCCCTGTCAGATCTGCACATGCCTCAGCGGGCGGAAGGTCAACTGCACAACGCAGCCCTGCCCCACGGCCAAAGCTCCCACGTGTGGCCTGTGTGAAGTAGCCCGCCTCCGCCAGAATGCAGACCAGTGCTGCCCCGAGTATGAGTGTGTGTGTGACCCAGTGAGCTGTGACCTGCCCCCAGTGCCTCACTGTGAACGTGGCCTCCAGCCCACACTGACCAACCCTGGCGAGTGCAGACCCAACTTCACaTGCGCCTGCAGGAAGGAGGAGTGCAAAAGAGTGTCCCCACCCTCCTGCCCCCCGCACCGTTTGCCCACCCTTCGGAAGACCCAGTGCTGTGATGAGTATGAGTGTGCCTGCAACTGTGTCAACTCCACAGTGAGCTGTCCCCTTGGGTACTTGGCCTCAACTGCCACCAATGACTGTGGCTGTACCACAACCACaTGCCTTCCCGACAAGGTGTGTGTCCACCGAAGCACCATCTACCCTGTGGGCCAGTTCTGGGAGGAGGGCTGCGATGTGTGCACaTGCACCGACATGGAGGATGCCGTGATGGGCCTCCGCGTGGCCCAGTGCTCCCAGAAGCCCTGTGAGGACAGCTGTCGGTCGGGCTTCACTTACGTTCTGCATGAAGGCGAGTGCTGTGGAAGGTGCCTGCCATCTGCCTGTGAGGTGGTGACTGGCTCACCGCGGGGGGACTCCCAGTCTTCCTGGAAGAGTGTCGGCTCCCAGTGGGCCTCCCCGGAGAACCCCTGCCTCATCAATGAGTGTGTCCGAGTGAAGGAGGAGGTCTTTATACAACAAAGGAACGTGTCCTGCCCCCAGCTGGAGGTCCCTGTCTGCCCCTCGGGCTTTCAGCTGAGCTGTAAGACCTCAGCGTGCTGCCCAAGCTGTCGCTGTGAGCGCATGGAGGCCTGCATGCTCAATGGCACTGTCATTGGGCCCGGGAAGACTGTGATGATCGATGTGTGCACGACCTGCCGCTGCATGGTGCAGGTcGGGGTCATCTCTGGATTCAAGCTGGAGTGCAGGAAGACCACaTGCAACCCCTGCCCCCTGGGTTACAAGGAAGAAAATAACACAGGTGAATGTTGTGGGAGATGTTTGCCTACGGCTTGCACCATTCAGCTAAGAGGAGGACAGATCATGACACTGAAGCGTGATGAAACGCTCCAGGATGGCTGTGATACTCACTTCTGCAAGGTCAATGAGAGAGGAGAGTACTTCTGGGAGAAGAGGGTCACAGGCTGCCCACCCTTTGATGAACACAAGTGTCTGGCTGAGGGAGGTAAAATTATGAAAATTCCAGGCACaTGCTGTGACACATGTGAGGAGCCTGAGTGCAACGACATCACTGCCAGGCTGCAGTATGTCAAGGTGGGAAGCTGTAAGTCTGAAGTAGAGGTGGATATCCACTACTGCCAGGGCAAATGTGCCAGCAAAGCCATGTACTCCATTGACATCAACGATGTGCAGGACCAGTGCTCCTGCTGCTCTCCGACACGGACGGAGCCCATGCAGGTcGCCCTGCACTGCACCAATGGCTCTGTTGTGTACCATGAGGTTCTCAATGCCATGGAGTGCAAATGCTCCCCCAGGAAGTGCAGCAAG'.upper())

In [42]:
# Define genes and check primers against them
genes = {
           'FRED1':FRED_SB1,
           'FRED2':FRED_SB2,
           'FRED3':FRED_SB3,
           'VWF1':VWF_SB1,
           'VWF2':VWF_SB2,
           'VWF3':VWF_SB3,
           'VWF4':VWF_SB4,
           'VWF5':VWF_SB5
        }

forward_primer_array = np.zeros((len(orthogonal_F['PrimerEnd']),len(genes)))
for j, genename in enumerate(genes.keys()):
    print('\n Processing forward primers against ' + genename)
    gene = genes[genename]
    for i, primer in enumerate(orthogonal_F['PrimerEnd']):
        forward_primer_array[i,j] = check_nonspecific(primer, gene)
        
reverse_primer_array = np.zeros((len(orthogonal_R['PrimerEnd']),len(genes)))
for j, genename in enumerate(genes.keys()):
    print('\n Processing reverse primers against ' + genename)
    gene = genes[genename]
    for i, primer in enumerate(orthogonal_R['PrimerEnd']):
        reverse_primer_array[i,j] = check_nonspecific(primer, gene)
        
# Compute number of nonspecific binding sites for each primer
orthogonal_F['Num_Nonspecific_Binding_Sites'] = np.sum(forward_primer_array, axis=1)
orthogonal_R['Num_Nonspecific_Binding_Sites'] = np.sum(reverse_primer_array, axis=1)



 Processing forward primers against FRED1
Found non-specific match at 1280bp:
 match:GCTTTCCAGTAGGATACACC
primer:AAGGCCCAGAAGGATACAAC Tm:20.43270409953766


/opt/miniconda3/lib/python3.13/site-packages/primer3/bindings.py:305: UserWarning: Function deprecated please use "calc_heterodimer" instead
  return THERMO_ANALYSIS.calcHeterodimer(



 Processing forward primers against FRED2
Found non-specific match using Primer3 at 1586bp:
 match:TAGCAGATGAGGAGAGGGCC
primer:GACCATGCAAGGAGAGGTAC Tm:19.7

 Processing forward primers against FRED3
Found non-specific match at 548bp:
 match:ATCTGCTCAGCATGGGCAGC
primer:AGTATCTCAGCAAGGGCAAC Tm:25.3

 Processing forward primers against VWF1
Found non-specific match at 1006bp:
 match:CTCCTGGATGAAGGCCTCTG
primer:AAAGCACTCTTAGGCCTCTG Tm:23.5
Found non-specific match at 1478bp:
 match:AAGGAGCGTTTCTGGCAGCC
primer:AATCAGTTTCTTTGGCAGCC Tm:22.038901619925298

 Processing forward primers against VWF2
Found non-specific match at 29bp:
 match:CCTGCGCCCTCAACCCGCGC
primer:AGTTGTAATATCACCCGCGC Tm:32.6
Found non-specific match at 1492bp:
 match:GCGCACGCCTCTCCCCGCGC
primer:AGTTGTAATATCACCCGCGC Tm:26.79564064854054
Found non-specific match at 330bp:
 match:CCGCTCTCTCTCTTACCCGG
primer:AATAGGAACCTCTTACGCGG Tm:23.0
Found non-specific match at 184bp:
 match:TGCCTGTGCGGCGCCCTGGC
primer:ACATTAAATTTCGCCGTGGC Tm

In [54]:
# Process primers
orthogonal_F_touse = orthogonal_F[(orthogonal_F.BsaI_Site_Present == False) & \
                                  (orthogonal_F.Num_Nonspecific_Binding_Sites == 0) & \
                                  (orthogonal_F.Exclude == False)][['Well Position','PrimerEnd','Worse']]
orthogonal_R_touse = orthogonal_R[(orthogonal_R.BsaI_Site_Present == False) & \
                                  (orthogonal_R.Num_Nonspecific_Binding_Sites == 0) & \
                                  (orthogonal_R.Exclude == False)][['Well Position','PrimerEnd','Worse']]

# Remove primer sequences manually - they bind to one of the genes on the gene list and don't pass oligo qc
primer_seqs_to_remove = ['AGTTGTAATATCACCCGCGC', 'ACAGAACGAACAGGCACTAC' ,'ATACTGTAAGAACCACGCGG']
orthogonal_F_touse_filt = orthogonal_F_touse[~orthogonal_F_touse['PrimerEnd'].isin(primer_seqs_to_remove)]
orthogonal_R_touse_filt = orthogonal_R_touse[~orthogonal_R_touse['PrimerEnd'].isin(primer_seqs_to_remove)]

In [55]:
# Import prevalidated primer combos
validated_primer_combos = pd.read_csv('./Validated Gene Primer Pair Combinations.csv')
validated_primer_combos

,Gene,Block,Validated F,Validated R
0,ARAFcterm,6,F1,A4
1,ARAFnterm,5,A11,A11
2,ARAFnterm,2,E10,F10
3,BRAFcterm,6,A4,A4
4,BRAFcterm,7,B4,B4
...,...,...,...,...
90,ERBB2cterm,2,B2,A7
91,ERBB2cterm,3,C2,B7
92,ERBB2cterm,4,D2,C7
93,ERBB2cterm,10,B3,A8


In [56]:
# Remove validated combos that are not usable since they bind to other genes
print('Forward wells validated but not usable:')
validated_not_usable_F = []
validated_not_usable_R = []
for f_well in np.unique(validated_primer_combos['Validated F'].values):
    if f_well not in orthogonal_F_touse_filt['Well Position'].values:
        print(f_well)
        validated_not_usable_F.append(f_well)
        
print('Reverse wells validated but not usable')
for r_well in np.unique(validated_primer_combos['Validated R'].values):
    if r_well not in orthogonal_R_touse_filt['Well Position'].values:
        print(r_well)
        validated_not_usable_R.append(r_well)

# filter out unusable primers
validated_primer_combos_filtered = validated_primer_combos[~validated_primer_combos['Validated F'].isin(validated_not_usable_F) & \
                                                           ~validated_primer_combos['Validated R'].isin(validated_not_usable_R)] \
                        .rename(columns={'Validated F':'Forward Name',
                                                 'Validated R':'Reverse Name'})

# add primer sequences
validated_primer_combos_filtered = pd.merge(validated_primer_combos_filtered,
         orthogonal_F[['Well Position','PrimerEnd']].rename(columns={'Well Position':'Forward Name',
                                                                     'PrimerEnd': 'Forward Primer'}), on='Forward Name')
validated_primer_combos_filtered = pd.merge(validated_primer_combos_filtered,
         orthogonal_R[['Well Position','PrimerEnd']].rename(columns={'Well Position':'Reverse Name',
                                                                     'PrimerEnd': 'Reverse Primer'}), on='Reverse Name')


Forward wells validated but not usable:
Reverse wells validated but not usable
B10
H7


In [57]:
# Generate more random combos of orthogonal F and reverse primers that are not "red" and that are not validated
# concatenate F and R primers into pandas dataframe
niters = 2
minlens_FR = min(len(orthogonal_F_touse_filt), len(orthogonal_R_touse_filt))
orthogonal_primers_iter = []
for i in range(niters):
    orthogonal_primers_iter.append(pd.concat([orthogonal_F_touse_filt.sample(n=minlens_FR).reset_index().\
                                                  drop(['index'], axis=1).rename(columns={'Well Position':'Forward Name',
                                                                                         'PrimerEnd':'Forward Primer'}),
                                              orthogonal_R_touse_filt.sample(n=minlens_FR).reset_index().\
                                                  drop(['index'], axis=1).rename(columns={'Well Position':'Reverse Name',
                                                                                         'PrimerEnd':'Reverse Primer'})],
                                                axis=1)\
                                    .drop(columns='Worse'))
orthogonal_primers_touse = pd.concat(orthogonal_primers_iter).drop_duplicates()
    
# remove any prevalidated combos
orthogonal_primers_touse = orthogonal_primers_touse[~orthogonal_primers_touse[['Forward Name','Reverse Name']]\
                                .apply(lambda a: (validated_primer_combos_filtered[['Forward Name','Reverse Name']] == a).all(1).any(),axis=1)]\
                                .reset_index(drop=True)
orthogonal_primers_touse.to_csv('orthogonal_primer_random_combos_used_DH_091125.csv')

In [58]:
#Import BsaI data
bsaI_empirical = pd.read_csv('bsaI_empirical.csv')
bsaI_empirical.index = bsaI_empirical['Overhang']
bsaI_empirical = bsaI_empirical.drop(columns=['Overhang'])
bsaI_empirical = bsaI_empirical + 1
bsaI_empirical

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
Overhang,,,,,,,,,,,,,,,,,,,,,
TTTT,636,9,41,17,3,1,1,1,8,1,...,1,1,1,1,1,1,1,1,1,1
GTTT,4,477,5,46,1,21,1,2,1,16,...,1,1,1,1,1,1,1,1,1,1
CTTT,2,2,597,3,1,1,19,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ATTT,9,5,2,643,1,1,1,7,1,2,...,1,1,1,1,1,1,1,1,1,1
TGTT,1,1,1,1,494,17,65,57,3,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACAA,1,1,1,1,1,1,1,1,1,1,...,1,1,11,3,8,480,1,1,1,1
TAAA,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,362,1,11,4
GAAA,1,1,1,1,1,1,1,1,1,1,...,1,1,1,3,1,1,6,716,2,20


In [59]:
#Import information about codon usage for mutagenesis
codons_ranked_by_usage = {
    "A": ["GCC", "GCT", "GCA", "GCG"],
    "C": ["TGC", "TGT"],
    "D": ["GAC", "GAT"],
    "E": ["GAG", "GAA"],
    "F": ["TTC", "TTT"],
    "G": ["GGC", "GGA", "GGG", "GGT"],
    "H": ["CAC", "CAT"],
    "I": ["ATC", "ATT", "ATA"],
    "K": ["AAG", "AAA"],
    "L": ["CTG", "CTC", "CTT", "TTG", "TTA", "CTA"],
    "M": ["ATG"],
    "N": ["AAC", "AAT"],
    "P": ["CCC", "CCT", "CCA", "CCG"],
    "Q": ["CAG", "CAA"],
    "R": ["CGG", "AGA", "AGG", "CGC", "CGA", "CGT"],
    "S": ["AGC", "TCC", "TCT", "AGT", "TCA", "TCG"],
    "T": ["ACC", "ACA", "ACT", "ACG"],
    "V": ["GTG", "GTC", "GTT", "GTA"],
    "W": ["TGG"],
    "Y": ["TAC", "TAT"],
}


In [60]:
# Make the blacklist of overhangs to exclude
# Tiny helper that works with either str or Seq
def rc(s: str) -> str:
    return str(Seq(str(s)).reverse_complement())

# Build blacklist 
manual_blacklist = [
    "CGCC","GCGG","GGCG","CGCG","CCGC","GGCC","CGGC","GGTG","CACC","CGCT","GCGC","GTCG","GCCG","AGCG",
    "CACG","GTGG","GGCT","GCTG","CGAC","GTTG","GTGC","GACG","AGCC","GGGC","ACGC","GCAC","CGTC",
    "CAGC","GGGG","GCGT","CGCA","GGTC","TGCG","GATG","GCCC","CGAG"
]

overhang_blacklist = manual_blacklist.copy()

# Ensure consistent casing
bsaI_empirical.index = bsaI_empirical.index.map(lambda x: str(x).upper())
bsaI_empirical.columns = bsaI_empirical.columns.map(lambda x: str(x).upper())

for codon in bsaI_empirical.index:
    codon_str = str(codon).upper()
    rc_str = rc(codon_str)

    # drop palindromes
    if codon_str == rc_str:
        overhang_blacklist.append(codon_str)
        continue

    # Look up efficiency
    if (codon_str in bsaI_empirical.index) and (rc_str in bsaI_empirical.columns):
        eff = bsaI_empirical.at[codon_str, rc_str]
        if eff < 300:
            overhang_blacklist.append(codon_str)
    else:
        print(f"Warning: no data for {codon_str} → {rc_str}")

# Deduplicate + sort
overhang_blacklist = sorted(set(overhang_blacklist))
print("Final overhang blacklist:", overhang_blacklist)

Final overhang blacklist: ['AATT', 'ACGC', 'ACGT', 'AGCC', 'AGCG', 'AGCT', 'ATAT', 'CAAC', 'CACC', 'CACG', 'CAGC', 'CATG', 'CCAC', 'CCCC', 'CCGC', 'CCGG', 'CGAC', 'CGAG', 'CGCA', 'CGCC', 'CGCG', 'CGCT', 'CGGC', 'CGTC', 'CTAG', 'GACG', 'GATC', 'GATG', 'GCAC', 'GCCC', 'GCCG', 'GCGC', 'GCGG', 'GCGT', 'GCTG', 'GGCC', 'GGCG', 'GGCT', 'GGGC', 'GGGG', 'GGTC', 'GGTG', 'GTAC', 'GTCG', 'GTGC', 'GTGG', 'GTTG', 'TATA', 'TCGA', 'TGCA', 'TGCG', 'TTAA']


In [61]:
# Set defaults
block_size_range = [153, 198]
max_oligo_size = 250
first_last_block_reduction = 8
slack = 5
randomsequencepad = "ACGCCGCCACGTGTTCGTTAACTGTTGATTGGTGGCACATAAGTAATACCATGGTCCCTGAAATTCGGCTCAGTTACTTCGAGCGTAATGTCTCAAATGGCGTAGAACGGCAATGACTGTTTGACACTAGGTGGTGTTCAGTTCGGTAACGGAGAGTCTGTGCGGCATTCTTATTAATACATTTGAAACGCGCCCAACTGACGCTAGGCAAGTCAGTGCAGGCTCCCGTGTTAGGATAAGGGTAAACATACAAGTCGATAGAAGATGGGTAGGGGCCTTCAATTCATCCAGCACTCTACG"


In [64]:
# 5p libraries
def post_qc(amp_primer_set, wt_oligos, primer_set, melt_temp_threshold = 35, check_all_primers=True):
    print("Running QC for primer specificity on WT oligos")
    f_primer_map = {}
    r_primer_map = {}
    # invert the primer to subpool map
    for k, v in amp_primer_set.items():
        f_primer_map[v[1]] = f_primer_map.get(v[1], []) + [k]
        r_primer_map[v[3]] = r_primer_map.get(v[3], []) + [k]
    
    # initialize list of nonspecific problems
    nonspecific = {}
    
    # add unused primers if check_all_primers
    if check_all_primers:
        all_f_primers = np.unique(primer_set['Forward Primer'])
        all_r_primers = np.unique(primer_set['Reverse Primer'])
        for f_primer in all_f_primers:
            if f_primer not in f_primer_map.keys():
                f_primer_map[f_primer] = []
        for r_primer in all_r_primers:
            if r_primer not in r_primer_map.keys():
                r_primer_map[r_primer] = []
        
    for f_primer, subpools_used in f_primer_map.items():
    # iterate over every barcode primer pair and match to each oligo to check for nonspecific amplification
        anneal_locs = []
        for subpoolcheck, fragmentcheck in wt_oligos.items():  # iterate over every WT oligo
            if (subpoolcheck not in subpools_used):  # ignore designed annealing (same name)
                if check_nonspecific(f_primer, fragmentcheck, Tm_rem = melt_temp_threshold, verbose=False) > 0: #use high Tm_rem
                    anneal_locs.append(subpoolcheck)
        if anneal_locs:
            nonspecific.update({f_primer:[a[0] + '_block' + str(a[1]+1) for a in anneal_locs]})
    for r_primer, subpools_used in r_primer_map.items():
    # iterate over every barcode primer pair and match to each oligo to check for nonspecific amplification
        anneal_locs = []
        for subpoolcheck, fragmentcheck in wt_oligos.items():  # iterate over every WT oligo
            if (subpoolcheck not in subpools_used):  # ignore designed annealing (same name)
                if check_nonspecific(r_primer, fragmentcheck, Tm_rem = melt_temp_threshold, verbose=False) > 0: #use high Tm_rem
                    anneal_locs.append(subpoolcheck)
        if anneal_locs:
            nonspecific.update({r_primer:[a[0] + '_block' + str(a[1]+1) for a in anneal_locs]})
    if nonspecific:
        print("Nonspecific Primers: (Manually removing primer sequence recommended)")
        print(nonspecific)
    else:
        print("No non-specific primers detected")
        
    return nonspecific

def build_kmers(sequence, 
                ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)

    return kmers

def compute_overlaps(breakpoints, 
                     inclusion_array, 
                     gene):
    
    overlaps = [[gene[val:val+4].reverse_complement(), gene[val:val+4]] for val in breakpoints]
    counter = 0
    for val in inclusion_array:
        if val == -1:
            (overlaps[counter][1],overlaps[counter+1][0]) = (overlaps[counter+1][0],overlaps[counter][1])
            counter += 1
        elif val == 0:
            overlaps[counter][1] = overlaps[counter+1][1]
            del overlaps[counter+1]
        
    return overlaps

def score_breakpoints(gene, 
                      breakpoint_pair, 
                      empirical, 
                      overhang_blacklist=overhang_blacklist):
    
    #subset empirical matrix by the set of all overlaps
    all_overlaps = []
    for breakpoint in breakpoint_pair:
        all_overlaps.append(gene[breakpoint:(breakpoint+4)])
        all_overlaps.append(gene[breakpoint:(breakpoint+4)].reverse_complement())
    all_overlaps = [str(o) for o in all_overlaps]
    if (len(np.unique(all_overlaps)) == len(all_overlaps)) & (len(set(all_overlaps).intersection(set(overhang_blacklist))) == 0):
        empirical_subset = empirical.loc[all_overlaps,all_overlaps]

        #compute fidelity score
        empirical_subset = empirical_subset/empirical_subset.sum(axis=1)
        fidelity_score = 1
        for breakpoint in breakpoint_pair:
            fidelity_score = fidelity_score * \
                empirical_subset.loc[str(gene[breakpoint:(breakpoint+4)]),
                                     str(gene[breakpoint:(breakpoint+4)].reverse_complement())]
    
    else:
        fidelity_score = 0
    
    return fidelity_score
    
def optimize_breakpoints(gene, 
                         breakpoint_pair, 
                         indices_to_shift, 
                         indices_of_array,
                         slack, 
                         empirical=bsaI_empirical, 
                         overhang_blacklist=overhang_blacklist):
    
    #compute all enrichments
    shifts = list(range(-slack,slack+1))
    if (len(indices_to_shift) > 2) | (len(indices_to_shift) < 1):
        print('Error -- too many or too few breakpoints!')
        optimum_breakpoint = breakpoint_pair
        optimum_score = 0
    elif (len(indices_to_shift) == 1): #external pair
        scores = [0]*len(shifts)
        for i,shift in enumerate(shifts):
            scores[i] = score_breakpoints(gene, breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shift] + breakpoint_pair[(indices_to_shift[0]+1):], 
                                          empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
        
        optimum_shift = np.argmax(scores)
        optimum_breakpoint = breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shifts[optimum_shift]] + breakpoint_pair[(indices_to_shift[0]+1):]
        optimum_score = scores[optimum_shift]
        optimum_length = optimum_breakpoint[indices_of_array[1]] - optimum_breakpoint[indices_of_array[0]]
            
            
    else: #internal pair
        indices_to_shift = sorted(indices_to_shift)
        scores = np.zeros((len(shifts),len(shifts)))
        for i,shift1 in enumerate(shifts):
            for j,shift2 in enumerate(shifts):
                scores[i,j] = score_breakpoints(gene, breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shift1] + \
                                                            breakpoint_pair[(indices_to_shift[0]+1):indices_to_shift[1]] + \
                                                            [breakpoint_pair[indices_to_shift[1]]+shift2] + breakpoint_pair[(indices_to_shift[1]+1):], 
                                              empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                
        optimum_shift = np.unravel_index(np.argmax(scores,axis=None), scores.shape)
        optimum_breakpoint = breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shifts[optimum_shift[0]]] + \
                                            breakpoint_pair[(indices_to_shift[0]+1):indices_to_shift[1]] + \
                                            [breakpoint_pair[indices_to_shift[1]]+shifts[optimum_shift[1]]] + breakpoint_pair[(indices_to_shift[1]+1):]
        optimum_score = scores[optimum_shift]
        optimum_length = optimum_breakpoint[indices_of_array[1]] - optimum_breakpoint[indices_of_array[0]] + 4
    
    return optimum_breakpoint, optimum_score, optimum_length

def optimize_gene(gene, 
                  max_tile_size,
                  first_last_block_reduction,
                  block_size_range=block_size_range, 
                  slack=slack, 
                  empirical=bsaI_empirical, 
                  overhang_blacklist=overhang_blacklist): 
    
    #setup initial inputs to optimization
    gene_size = len(gene)
    protein_size = len(gene.translate())
        
    #exclude gene if it is too big
    if protein_size > 1000:
        print('Protein size too big!')
        
    #divide genes between 500 and 1000aa into two blocks
    elif protein_size > 620:
        print('Protein size too big! Will add two superblock (620aa+ proteins) soon.')
        
    else:
        #gene is one superblock 
        #print('Protein is one superblock.')
        block_size = block_size_range[0] + np.argmin(
            [abs((gene_size+2*first_last_block_reduction)/(i+block_size_range[0]) - \
                     round((gene_size+2*first_last_block_reduction)/(i+block_size_range[0]))) \
                 for i in range(0, block_size_range[1]-block_size_range[0])])
        
        
        # now, set initial breakpoints
        fragment_number = int((gene_size+2*first_last_block_reduction)/block_size)
        
        # if any of the tiles are too big?
        tile_lengths = [1000]
        while max(tile_lengths) > (max_tile_size-2*slack):
            fragment_number = fragment_number + 1
            first_breakpoint = 0
            last_breakpoint = gene_size-4
            step = (last_breakpoint - first_breakpoint + 2*first_last_block_reduction)/fragment_number
            evenly_spaced_floats = [first_breakpoint] + [step * i - first_last_block_reduction for i in range(1,fragment_number)] + [last_breakpoint]        
            initial_breakpoints = [[first_breakpoint,int(evenly_spaced_floats[1])+slack+2,last_breakpoint]] + \
                                    [[0,int(evenly_spaced_floats[i])-2-slack,int(evenly_spaced_floats[i+1])+slack+2,last_breakpoint] for i in range(1,len(evenly_spaced_floats)-2)] + \
                                    [[0,int(evenly_spaced_floats[-2])-2-slack,last_breakpoint]]
            tile_lengths = [int(evenly_spaced_floats[1])+slack+2-first_breakpoint+first_last_block_reduction+4] + \
                                    [int(evenly_spaced_floats[i+1])-int(evenly_spaced_floats[i])+2*(slack+2)+4 for i in range(1,len(evenly_spaced_floats)-2)] + \
                                    [last_breakpoint+4-int(evenly_spaced_floats[-2])+slack+2+first_last_block_reduction]
            

        #optimize each breakpoint
        optimum_breakpoints = []
        optimum_scores = []
        optimum_lengths = []
        oligo_array_indices = []
        for k,breakpoint in enumerate(initial_breakpoints):
            if len(breakpoint) == 3:
                indices_of_array = [0, 1] if k==0 else [1, 2]
                optimum_breakpoint, optimum_score, optimum_length = optimize_breakpoints(gene, breakpoint, [1], indices_of_array,
                                                            slack, empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                optimum_breakpoints.append(optimum_breakpoint)
                optimum_scores.append(optimum_score)
                optimum_lengths.append(optimum_length)
                oligo_array_indices.append(indices_of_array)
            else:
                indices_of_array = [1, 2]
                optimum_breakpoint, optimum_score, optimum_length = optimize_breakpoints(gene, breakpoint, [1, 2], indices_of_array,
                                                            slack, empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                optimum_breakpoints.append(optimum_breakpoint)
                optimum_scores.append(optimum_score)
                optimum_lengths.append(optimum_length)
                oligo_array_indices.append(indices_of_array)
    
    optimum_overlaps = [[str(gene[t:(t+4)]) for t in s] for s in optimum_breakpoints]
    if all([s >= 0.95 for s in optimum_scores]):
        print('All regions are high fidelity!')
    elif all([s >= 0.9 for s in optimum_scores]):
        print('Some regions are medium fidelity.')
    else:
        print('Some regions are low fidelity. Look closer')
        
    return optimum_breakpoints, optimum_overlaps, optimum_scores, optimum_lengths, oligo_array_indices

def generate_primer(DNA_seq,
                     Fwd=True,
                     extendtoCG=False,
                     smallest_primer_size=16,
                     largest_primer_size=30,
                     Tm=55):
    
    #Setup melting temperature arrays
    melt_temp_array = np.zeros(largest_primer_size-smallest_primer_size+1)
    
    if Fwd:
        DNA_seq_touse = DNA_seq
    else:
        DNA_seq_touse = DNA_seq.reverse_complement()
            
    #Make melting temperature arrays
    primer_length = 0
    for i in range(smallest_primer_size,largest_primer_size+1):
        melt_temp_array[i-smallest_primer_size] = mt.Tm_NN(DNA_seq_touse[0:i])
        
        #Pick F primer when Tm is first >F_Tm
        if (melt_temp_array[i-smallest_primer_size] >= Tm) & (primer_length==0):
            primer_length = i
    
    #If Tm isnt high enough after max bases, just set primer length to be max and hope it works
    if (primer_length == 0):
        primer_length = largest_primer_size
        
    if extendtoCG:
        while ((DNA_seq_touse[primer_length-1] == 'A') | (DNA_seq_touse[primer_length-1] == 'T')) & \
                    (primer_length < largest_primer_size):
            primer_length += 1
    
    return DNA_seq_touse[0:primer_length]

def make_mutations(region_name,
                       region,
                       region_flanks=[Seq(''),Seq('')],
                       nt_start=0, #zero-indexed!
                       wt_only=False,
                       synonymous=True,
                       stops='TAA',
                       all3ntdeletions=True,
                       mutation_list=False,
                       codons_ranked_by_usage=codons_ranked_by_usage,
                       aa_start=0):
                       
    oligo_array = {}
    #Check that region has size divisible by three
    if (len(region)/3 != len(region)//3) | (nt_start/3 != nt_start//3):
        print('Region is not translatable!')
        
    else:
        #add wt seq to oligo array
        oligo_name = region_name + '_WT'
        wt_seq = \
            region_flanks[0] + region + region_flanks[1]
        oligo_array[oligo_name] = wt_seq
        
        if not wt_only:
            
            # see if a mutation list was given
            if mutation_list == False:
                    
                #loop over amino acids
                for j in range(0,len(region),3):

                    #add all missense variants
                    aa = region[j:(j+3)].translate()
                    for aa_to in codons_ranked_by_usage.keys():
                        if aa_to != aa:
                            oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + str(aa_to)
                            seq_to_append = \
                                region_flanks[0] + \
                                region[0:j] + Seq(codons_ranked_by_usage[aa_to][0]) + \
                                region[(j+3):] + \
                                region_flanks[1]
                            oligo_array[oligo_name] = seq_to_append

                    #add synonymous variant if True and if possible, 
                    # using the most common codon that is NOT the codon in the gene
                    if synonymous:
                        if len(codons_ranked_by_usage[aa]) > 1:
                            oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + str(aa)
                            possible_codons = codons_ranked_by_usage[aa].copy()
                            possible_codons.remove(region[j:(j+3)])
                            seq_to_append = \
                                region_flanks[0] + \
                                region[0:j] + Seq(possible_codons[0]) + \
                                region[(j+3):] + \
                                region_flanks[1]
                            oligo_array[oligo_name] = seq_to_append

                    #add stops if true
                    if stops:
                        oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + 'X'
                        seq_to_append = \
                            region_flanks[0] + \
                            region[0:j] + Seq(stops) + \
                            region[(j+3):] + \
                            region_flanks[1]
                        oligo_array[oligo_name] = seq_to_append

                    #add all 3nt deletions if True
                    if all3ntdeletions:
                        for k in range(0,3):
                            if j+k+3 <= len(region):
                                oligo_name = region_name + '_' + 'del' + str(nt_start+j+k+1+3*aa_start)
                                seq_to_append = \
                                    region_flanks[0] + \
                                    region[0:(j+k)] + \
                                    region[(j+k+3):] + \
                                    region_flanks[1]
                                oligo_array[oligo_name] = seq_to_append
                                
            else:
                
                #loop over mutation list
                for i in range(len(mutation_list)):
                        
                    #iterate over every single aa change
                    oligo_name = region_name + '_' + 'variant' + str(i+1)
                    seq_to_append = region
                    for k,v in enumerate(mutation_list[i]):
                        aa_from = v[0]
                        aa_to = v[-1]
                        pos = int(v[1:-1])
                        j=3*(pos-(nt_start//3+1))
                        aa = region[j:(j+3)].translate()
                        if aa != aa_from:
                            print('Check mutation list!')
                        else:
                            seq_to_append = \
                                seq_to_append[0:j] + \
                                Seq(codons_ranked_by_usage[aa_to][0]) + \
                                seq_to_append[(j+3):]

                    # append oligo to array
                    seq_to_append = region_flanks[0] + \
                                    seq_to_append + \
                                    region_flanks[1]
                    oligo_array[oligo_name] = seq_to_append
        
    return oligo_array


def write_oligo_library(genes,
                        oligo_file='./oligo_test.csv',
                        primer_file='./primer_test.tsv',
                        gbl_file='./gbl_test.tsv',
                        gbl_large_file='./gbl_test_large.tsv',
                        amp_primer_key_file='./amp_primer_key.tsv',
                        breakpoint_file='./breakpoints.tsv',
                        primer_set=orthogonal_primers_touse,
                        codons_ranked_by_usage=codons_ranked_by_usage,
                        block_size_range=block_size_range, 
                        max_oligo_size=max_oligo_size,
                        slack=slack, 
                        empirical=bsaI_empirical, 
                        overhang_blacklist=overhang_blacklist,
                        validated_primer_set=False,
                        aa_start=False,
                        wt_only=False,
                        synonymous=True,
                        stops='TAA',
                        all3ntdeletions=True,
                        mutations_to_use=False,
                        find_pcr_primers=True,
                        smallest_primer_size=16,
                        largest_primer_size=30,
                        Tm=55,
                        extendtoCG=True,
                        bsaI_firstoverlap='ACGA',
                        bsaI_lastoverlap='TTTC',
                        all_blocks=True,
                        blocks_to_include=False,
                        tile_boundaries=False,
                        paqcIcapF=True,
                        paqcIcapR=True,
                        check_all_primers=True,
                        qc_melt_temp_threshold=32,
                        gblock_min_size=300,
                        gblock_large_threshold=1000,
                        randomsequencepad=randomsequencepad):
    
    #Split up primer set into F and R primers, cannot do more than 82 sublibraries
    oligo_primer_counter = 0
    oligo_array = {}
    amp_primers = {}
    gblocks = {}
    num_blocks = {}
    amp_primer_dict = {}
    breakpoint_dict = {}
    
    #Convert genes to Seq and genes to list
    gene_names = list(genes.keys())
    genes = [Seq(genes[gene_name].upper()) for gene_name in gene_names]
    
    #PaqCI and BsaI site sequences and overhangs for paqcI
    paqcI_seq = Seq('CACCTGC')
    paqcI_overhang_nterm_ctag = Seq('CCAC')
    paqcI_overhang_cterm_ctag = Seq('CGGG')
    paqcI_overhang_nterm_ntag = Seq('TGGC')
    paqcI_overhang_cterm_ntag = Seq('TAGG')
    paqcI_seqplusfour = Seq('CACCTGCCTAG')
    bsaI_seq = Seq('GGTCTC')
    bsaI_seqplusone = Seq('GGTCTCT')
    pcr_capseq = Seq('GGCTAC') + bsaI_seqplusone
    gbl_capseq_F = Seq('CCGCGTGATTACGAGTCG') + pcr_capseq
    gbl_capseq_R = Seq('GGGTTAGCAAGTGGCAGCCT') + pcr_capseq
    
    # set max size of a tile
    primer_len = len(primer_set['Forward Primer'][0])
    max_tile_size = max_oligo_size - 2*primer_len - 2*len(bsaI_seqplusone)
    first_last_block_reduction = len(paqcI_seqplusfour)
    
    # iterate over genes
    for r,gene in enumerate(genes):
        
        gene_name = gene_names[r]
        print('Processing gene ' + str(r+1) + ': ' + gene_name)
        
        # amino acid to start at
        if aa_start != False:
            if gene_name in aa_start.keys():
                aa_start_gene=aa_start[gene_name]-1
            else:
                aa_start_gene=0
        else:
            aa_start_gene=0
        
        #exclude if gene size is not divisible by three
        if len(gene)/3 != len(gene)//3:
            print('Gene length is not divisible by 3!')
    
        #exclude if there is a paqcI site in the gene
        elif any([True for kmer in build_kmers(gene, len(paqcI_seq)) if kmer==paqcI_seq]) | \
            any([True for kmer in build_kmers(gene.reverse_complement(), len(paqcI_seq)) if kmer==paqcI_seq]):
            print('Gene has paqcI site!')
        
        #exclude if there is a BsaI site in the gene
        elif any([True for kmer in build_kmers(gene, len(bsaI_seq)) if kmer==bsaI_seq]) | \
            any([True for kmer in build_kmers(gene.reverse_complement(), len(bsaI_seq)) if kmer==bsaI_seq]):
            print('Gene has BsaI site!')
            
        else:
            print('Gene has no paqcI or BsaI sites! Performing GoldenGate optimization...')
            
            #cap gene with BsaI breakpoints and possible paqcI sites 
            if paqcIcapF & paqcIcapR:
                gene_capped = bsaI_firstoverlap + paqcI_seqplusfour +  paqcI_overhang_nterm_ntag + \
                        gene + paqcI_overhang_cterm_ntag + paqcI_seqplusfour.reverse_complement() + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap + paqcI_seqplusfour + paqcI_overhang_nterm_ntag)
                capping_length_R = len(paqcI_overhang_cterm_ntag + paqcI_seqplusfour.reverse_complement() + \
                                   bsaI_lastoverlap)
            elif paqcIcapF:
                gene_capped = bsaI_firstoverlap + paqcI_seqplusfour + paqcI_overhang_nterm_ntag + \
                        gene + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap + paqcI_seqplusfour + paqcI_overhang_nterm_ntag)
                capping_length_R = len(bsaI_lastoverlap)
            elif paqcIcapR:
                gene_capped = bsaI_firstoverlap + \
                        gene + paqcI_overhang_cterm_ntag + paqcI_seqplusfour.reverse_complement() + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap)
                capping_length_R = len(paqcI_overhang_cterm_ntag + paqcI_seqplusfour.reverse_complement() + \
                                   bsaI_lastoverlap)
            else:
                gene_capped = bsaI_firstoverlap + gene + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap)
                capping_length_R = len(bsaI_lastoverlap)
            
            #Optimize gene if tile boundaries are not given
            if tile_boundaries is False:
                optimum_breakpoints, optimum_overlaps, optimum_scores, optimum_lengths, oligo_array_indices = \
                optimize_gene(gene_capped, 
                          max_tile_size=max_tile_size,
                          first_last_block_reduction=first_last_block_reduction,
                          block_size_range=block_size_range, 
                          slack=slack, 
                          empirical=bsaI_empirical, 
                          overhang_blacklist=overhang_blacklist)
                pprint.pprint({'Optimum Breakpoints': optimum_breakpoints, 
                       'Optimum Overlaps': optimum_overlaps, 
                       'Optimum Scores': optimum_scores})
                num_blocks[gene_name] = len(optimum_breakpoints)
            else:
                optimum_breakpoints = tile_boundaries[gene_name]
                if len(optimum_breakpoints[0])==4:
                    if len(optimum_breakpoints)>1:
                        #multiple tiles, including one at beginning of gene
                        oligo_array_indices = [[0,1]] + [[1,2]]*(len(optimum_breakpoints)-1)
                    else:
                        # one tile
                        if ((optimum_breakpoints[1]-optimum_breakpoints[0]) > (optimum_breakpoints[2]-optimum_breakpoints[1])):
                            # at end of gene
                            oligo_array_indices = [[1,2]]
                        else:
                            # at beginning of gene
                            oligo_array_indices = [[0,1]]
                else:
                    # multiple tiles, starting in the middle
                    oligo_array_indices = [[1,2]]*(len(optimum_breakpoints))
                num_blocks[gene_name] = len(optimum_breakpoints)
                
            
            #add primers for gene_F and gene_R that are repeated constantly throughout the PCRs
            #note: should probably prevalidate these primers!
            if find_pcr_primers:
                F_primer = generate_primer(gene,
                                           Fwd=True,
                                           extendtoCG=extendtoCG,
                                           smallest_primer_size=smallest_primer_size,
                                           largest_primer_size=largest_primer_size,
                                           Tm=Tm)
                F_primer = pcr_capseq + bsaI_firstoverlap + paqcI_seqplusfour + F_primer
                amp_primers[gene_name+'_gene'+'_ampF'] = F_primer
                R_primer = generate_primer(gene,
                                           Fwd=False,
                                           extendtoCG=extendtoCG,
                                           smallest_primer_size=smallest_primer_size,
                                           largest_primer_size=largest_primer_size,
                                           Tm=Tm)
                R_primer = pcr_capseq + Seq(bsaI_lastoverlap).reverse_complement() + \
                            paqcI_seqplusfour + R_primer
                amp_primers[gene_name+'_gene'+'_ampR'] = R_primer
            
            #make oligos, primers, gblocks for each block
            for i,breakpoint in enumerate(optimum_breakpoints):
                
                #find indices of breakpoint that correspond to oligo vs need to be PCRed/gblock
                pcr_indices = [[j,j+1] for j in range(len(breakpoint)-1)]
                pcr_indices.remove(oligo_array_indices[i])
                
                #find mutagenic window of oligo
                oligo_breaks = [breakpoint[j] for j in oligo_array_indices[i]]
                oligo_mutagenic_window = [int(3*np.ceil(max(oligo_breaks[0]+4-capping_length_F,3)/3)), int(3*np.floor(min(oligo_breaks[1]-capping_length_F,len(gene)-1)/3))]
                
                #subset the right block if needed
                if (all_blocks == True) | ((i+1) in blocks_to_include[r] if blocks_to_include != False else True): #subset on allowed blocks
                
                    #add pcr primers and gblocks, one segment at a time
                    for k,pcr_index in enumerate(pcr_indices):
                        piece_name = gene_name + '_block' + str(i+1) + '_s' + str(k+1)
                        pcr_breaks = [breakpoint[j] for j in pcr_index]
                                            
                        #get pcr primers
                        if find_pcr_primers:
                            if pcr_breaks[0] == breakpoint[0]: #Fragment beginning at gene start 
                                R_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=False,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                R_primer = pcr_capseq + R_primer
                                amp_primers[piece_name+'_ampR'] = R_primer
                            elif pcr_breaks[1] == breakpoint[-1]: #Fragment ending at gene end
                                F_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=True,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                F_primer = pcr_capseq + F_primer
                                amp_primers[piece_name+'_ampF'] = F_primer
                            else:
                                F_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=True,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                F_primer = pcr_capseq + F_primer
                                R_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=False,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                R_primer = pcr_capseq + R_primer
                                amp_primers[piece_name+'_ampF'] = F_primer
                                amp_primers[piece_name+'_ampR'] = R_primer

                        #make gblocks
                        gbl = gbl_capseq_F + gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)] + gbl_capseq_R.reverse_complement()
                        gblocks[piece_name] = gbl
                        
                    #add oligos to oligo array, checking first for validated primers if they are given
                    validated=False
                    if validated_primer_set is not False:
                        validated_combo = validated_primer_set.query('Gene == @gene_name & Block == (@i+1)')
                        if not validated_combo.empty:
                            validated=True
                            name_primer_F, primer_F, name_primer_R, primer_R = \
                                validated_combo[['Forward Name',
                                                  'Forward Primer',
                                                  'Reverse Name',
                                                  'Reverse Primer']].values[0]
                        else:
                            name_primer_F, primer_F, name_primer_R, primer_R = \
                                primer_set.iloc[oligo_primer_counter,][['Forward Name',
                                                                      'Forward Primer',
                                                                      'Reverse Name',
                                                                      'Reverse Primer']]
                            oligo_primer_counter += 1
                    else:
                        name_primer_F, primer_F, name_primer_R, primer_R = \
                                primer_set.iloc[oligo_primer_counter,][['Forward Name',
                                                                      'Forward Primer',
                                                                      'Reverse Name',
                                                                      'Reverse Primer']]
                        oligo_primer_counter += 1
                        
                    # if mutations are given, use those - otherwise make all mutations or wtonly
                    if mutations_to_use == False:
                        add_on_array = make_mutations(gene_name + '_block' + str(i+1),
                                           gene[oligo_mutagenic_window[0]:oligo_mutagenic_window[1]],
                                           region_flanks=[Seq(primer_F) + \
                                                          bsaI_seqplusone + \
                                                          gene_capped[oligo_breaks[0]:(oligo_mutagenic_window[0]+capping_length_F)] ,
                                                          gene_capped[(oligo_mutagenic_window[1]+capping_length_F):(oligo_breaks[1]+4)] + \
                                                          bsaI_seqplusone.reverse_complement() + \
                                                          Seq(primer_R).reverse_complement()],
                                           nt_start=oligo_mutagenic_window[0],
                                           wt_only=wt_only,
                                           synonymous=synonymous,
                                           stops=stops,
                                           all3ntdeletions=all3ntdeletions,
                                           codons_ranked_by_usage=codons_ranked_by_usage,
                                           aa_start=aa_start_gene)
                    else:
                        add_on_array = make_mutations(gene_name + '_block' + str(i+1),
                                           gene[oligo_mutagenic_window[0]:oligo_mutagenic_window[1]],
                                           region_flanks=[Seq(primer_F) + \
                                                          bsaI_seqplusone + \
                                                          gene_capped[oligo_breaks[0]:(oligo_mutagenic_window[0]+capping_length_F)] ,
                                                          gene_capped[(oligo_mutagenic_window[1]+capping_length_F):(oligo_breaks[1]+4)] + \
                                                          bsaI_seqplusone.reverse_complement() + \
                                                          Seq(primer_R).reverse_complement()],
                                           nt_start=oligo_mutagenic_window[0],
                                           mutation_list=mutations_to_use[(gene_name,i+1)],
                                           codons_ranked_by_usage=codons_ranked_by_usage,
                                           aa_start=aa_start_gene)
                    oligo_array.update(add_on_array)
                    amp_primer_dict.update({(gene_name,i+1): (name_primer_F,primer_F,name_primer_R,primer_R,validated)})
                    breakpoint_dict.update({(gene_name,i+1): oligo_mutagenic_window})
                    
    #Check that max oligo is less than the max oligo length
    if sum([len(s)>max_oligo_size for s in oligo_array.values()]) == 0:
        print('All oligos are below the maximum 250bp!')
    else:
        print('Some oligos are TOO BIG!')
        
    #Check for nonspecific amplification
    wt_oligos = {tuple([key.split('_')[0],
                        int((key.split('_block')[1]).split('_')[0])]
                      ):oligo_array[key] \
                     for key in oligo_array.keys() if 'WT' in key}
    nonspecific_primers = post_qc(amp_primer_dict, 
                                  wt_oligos,
                                  primer_set, 
                                  melt_temp_threshold=qc_melt_temp_threshold,
                                  check_all_primers=check_all_primers)
    
    # Check that all mutagenic windows overlap
    breakpoint_df = pd.DataFrame.from_dict(breakpoint_dict, orient='index', columns=['Mutagenesis Start','Mutagenesis End'])
    breakpoint_df.index = pd.MultiIndex.from_tuples(breakpoint_dict.keys())
    breakpoint_df = breakpoint_df.reset_index().rename(columns={'level_0':'Gene',
                                                                'level_1':'Block'})
    if (mutations_to_use == False) and (all_blocks == True):
        missed_counter = 0
        for r,gene_group_breakpoints in breakpoint_df.groupby('Gene'):
            for k,row in gene_group_breakpoints.iterrows():
                # look at whether the current row start is later than the last row end
                if row['Block'] > 1:
                    if row['Mutagenesis Start'] > end:
                        missed_counter += 1
                        print('Mutagenic window missed at ' + str(r) + ' block ' + str(row['Block']))
                start,end = row['Mutagenesis Start'],row['Mutagenesis End']
        if missed_counter == 0:
            print('All mutagenic windows overlap!')
        else:
            print(str(missed_counter) + ' number of times the mutagenic window does not close!')
                
    #Remove any oligos with additional BsaI sites or paqcI sites
    bad_oligos = []
    for name,oligo in oligo_array.items():
        #check for paqcI sites
        paqcI_F = sum([True for kmer in build_kmers(oligo, len(paqcI_seq)) if kmer==paqcI_seq])
        paqcI_R = sum([True for kmer in build_kmers(oligo.reverse_complement(), len(paqcI_seq)) if kmer==paqcI_seq])
        #check that oligo is block 1 if it contains a paqcI site in the forward orientation 
        if paqcI_F > 0:
            if ('block1' not in name) | (paqcI_F > 1):
                bad_oligos.append(name)
        #check that the oligo is block final if it contains a paqcI site in the reverse orientation
        if paqcI_R > 0:
            if ('block'+str(num_blocks[name.split('_')[0]]) not in name) | (paqcI_R > 1):
                bad_oligos.append(name)
        #check for more than one BsaI site
        bsaI_F = sum([True for kmer in build_kmers(oligo, len(bsaI_seq)) if kmer==bsaI_seq])
        bsaI_R = sum([True for kmer in build_kmers(oligo.reverse_complement(), len(bsaI_seq)) if kmer==bsaI_seq])
        if (bsaI_F != 1) | (bsaI_R != 1):
            bad_oligos.append(name)
    bad_oligos=np.unique(bad_oligos)
    for oligo_name in bad_oligos:
        del oligo_array[oligo_name]
    print(str(len(bad_oligos)) + ' oligos deleted due to errant restriction sites.')
    
    #Remove any duplicate oligos
    new_dict = {}
    seen_values = set()
    counter=0
    for key, value in oligo_array.items():
        if value not in seen_values:
            new_dict[key] = value
            seen_values.add(value)
        else:
            counter += 1
    print(str(counter) + ' oligos removed due to duplication.')
    oligo_array = new_dict
    del new_dict
    
    #write oligo array to file
    with open(oligo_file, 'w') as f:
        for key in oligo_array.keys():
            f.write("%s,%s\n"%(key,oligo_array[key]))
    f.close()
            
    #write primers to file
    if find_pcr_primers:
        primer_order_sheet = []
        for key in amp_primers.keys():
            primer_order_sheet.append(key + '\t' + \
                     str(amp_primers[key]) + \
                     '\t' + '25nm' + '\t' + 'STD\n')
        print(*primer_order_sheet)
        with open(primer_file, 'w') as f:
            for line in primer_order_sheet:
                f.write(line)
        f.close()
    
    #write amplification primer key to file
    amp_primer_key = ['Gene' + '\t' + 'Block' + '\t' + \
                      'Forward Primer Well' + '\t' + 'Forward Primer' + '\t' + \
                      'Reverse Primer Well' + '\t' + 'Reverse Primer' + '\t' + 'Validated' + '\n']
    for key in amp_primer_dict.keys():
        genename, geneblock = key[0], str(key[1])
        name_primer_F, primer_F, name_primer_R, primer_R, validated = amp_primer_dict[key]
        amp_primer_key.append(genename + '\t' + geneblock + '\t' + \
                 name_primer_F + '\t' + primer_F + '\t' + \
                 name_primer_R + '\t' + primer_R + '\t' + str(validated) + '\n')
    print(*amp_primer_key)
    with open(amp_primer_key_file, 'w') as f:
        for line in amp_primer_key:
            f.write(line)
    f.close()
    
    #write breakpoint dict to file
    breakpoint_df.to_csv(breakpoint_file, sep='\t')
    
    #write gblocks to file
    gblock_order_sheet = []
    gblock_large_order_sheet = []
    for key in gblocks.keys():
        # pad gblock if it is not 300bp for Twist
        if len(gblocks[key]) < gblock_min_size:
            gblocks[key] = Seq(randomsequencepad[0:(gblock_min_size-len(gblocks[key]))]) + gblocks[key]
        if len(gblocks[key]) < gblock_large_threshold:
            gblock_order_sheet.append(key + '\t' + \
                     str(gblocks[key]) + '\n')
        else:
            gblock_large_order_sheet.append(key + '\t' + \
                     str(gblocks[key]) + '\n')
    print(*gblock_order_sheet)
    print(*gblock_large_order_sheet)
    with open(gbl_file, 'w') as f:
        for line in gblock_order_sheet:
            f.write(line)
    f.close()
    with open(gbl_large_file, 'w') as f:
        for line in gblock_large_order_sheet:
            f.write(line)
    f.close()
    
    return oligo_array,amp_primers,gblocks,amp_primer_dict,breakpoint_df
                

In [39]:
# Functions for internal superblock sets
def post_qc(amp_primer_set, wt_oligos, primer_set, melt_temp_threshold = 35, check_all_primers=True):
    print("Running QC for primer specificity on WT oligos")
    f_primer_map = {}
    r_primer_map = {}
    # invert the primer to subpool map
    for k, v in amp_primer_set.items():
        f_primer_map[v[1]] = f_primer_map.get(v[1], []) + [k]
        r_primer_map[v[3]] = r_primer_map.get(v[3], []) + [k]
    
    # initialize list of nonspecific problems
    nonspecific = {}
    
    # add unused primers if check_all_primers
    if check_all_primers:
        all_f_primers = np.unique(primer_set['Forward Primer'])
        all_r_primers = np.unique(primer_set['Reverse Primer'])
        for f_primer in all_f_primers:
            if f_primer not in f_primer_map.keys():
                f_primer_map[f_primer] = []
        for r_primer in all_r_primers:
            if r_primer not in r_primer_map.keys():
                r_primer_map[r_primer] = []
        
    for f_primer, subpools_used in f_primer_map.items():
    # iterate over every barcode primer pair and match to each oligo to check for nonspecific amplification
        anneal_locs = []
        for subpoolcheck, fragmentcheck in wt_oligos.items():  # iterate over every WT oligo
            if (subpoolcheck not in subpools_used):  # ignore designed annealing (same name)
                if check_nonspecific(f_primer, fragmentcheck, Tm_rem = melt_temp_threshold, verbose=False) > 0: #use high Tm_rem
                    anneal_locs.append(subpoolcheck)
        if anneal_locs:
            nonspecific.update({f_primer:[a[0] + '_block' + str(a[1]+1) for a in anneal_locs]})
    for r_primer, subpools_used in r_primer_map.items():
    # iterate over every barcode primer pair and match to each oligo to check for nonspecific amplification
        anneal_locs = []
        for subpoolcheck, fragmentcheck in wt_oligos.items():  # iterate over every WT oligo
            if (subpoolcheck not in subpools_used):  # ignore designed annealing (same name)
                if check_nonspecific(r_primer, fragmentcheck, Tm_rem = melt_temp_threshold, verbose=False) > 0: #use high Tm_rem
                    anneal_locs.append(subpoolcheck)
        if anneal_locs:
            nonspecific.update({r_primer:[a[0] + '_block' + str(a[1]+1) for a in anneal_locs]})
    if nonspecific:
        print("Nonspecific Primers: (Manually removing primer sequence recommended)")
        print(nonspecific)
    else:
        print("No non-specific primers detected")
        
    return nonspecific

def build_kmers(sequence, 
                ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)

    return kmers

def compute_overlaps(breakpoints, 
                     inclusion_array, 
                     gene):
    
    overlaps = [[gene[val:val+4].reverse_complement(), gene[val:val+4]] for val in breakpoints]
    counter = 0
    for val in inclusion_array:
        if val == -1:
            (overlaps[counter][1],overlaps[counter+1][0]) = (overlaps[counter+1][0],overlaps[counter][1])
            counter += 1
        elif val == 0:
            overlaps[counter][1] = overlaps[counter+1][1]
            del overlaps[counter+1]
        
    return overlaps

def score_breakpoints(gene, breakpoint_pair, empirical, overhang_blacklist=None):
    """
    empirical: a DataFrame indexed by 4-mers, columns are reverse complements
    overhang_blacklist: (optional) list of bad overhangs to penalize
    """
    fidelity_score = 1.0
    for bp in breakpoint_pair:
        kmer_seq   = gene[bp:bp+4]
        kmer       = str(kmer_seq)
        rev_kmer   = str(kmer_seq.reverse_complement())
        try:
            fidelity_score *= empirical.at[kmer, rev_kmer]
        except KeyError:
            fidelity_score = 0.0
            break
    return fidelity_score

    
def optimize_breakpoints(gene, 
                         breakpoint_pair, 
                         indices_to_shift, 
                         indices_of_array,
                         slack, 
                         empirical=bsaI_empirical, 
                         overhang_blacklist=overhang_blacklist):
    
    #compute all enrichments
    shifts = list(range(-slack,slack+1))
    if (len(indices_to_shift) > 2) | (len(indices_to_shift) < 1):
        print('Error -- too many or too few breakpoints!')
        optimum_breakpoint = breakpoint_pair
        optimum_score = 0
    elif (len(indices_to_shift) == 1): #external pair
        scores = [0]*len(shifts)
        for i,shift in enumerate(shifts):
            scores[i] = score_breakpoints(gene, breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shift] + breakpoint_pair[(indices_to_shift[0]+1):], 
                                          empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
        
        optimum_shift = np.argmax(scores)
        optimum_breakpoint = breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shifts[optimum_shift]] + breakpoint_pair[(indices_to_shift[0]+1):]
        optimum_score = scores[optimum_shift]
        optimum_length = optimum_breakpoint[indices_of_array[1]] - optimum_breakpoint[indices_of_array[0]]
            
            
    else: #internal pair
        indices_to_shift = sorted(indices_to_shift)
        scores = np.zeros((len(shifts),len(shifts)))
        for i,shift1 in enumerate(shifts):
            for j,shift2 in enumerate(shifts):
                scores[i,j] = score_breakpoints(gene, breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shift1] + \
                                                            breakpoint_pair[(indices_to_shift[0]+1):indices_to_shift[1]] + \
                                                            [breakpoint_pair[indices_to_shift[1]]+shift2] + breakpoint_pair[(indices_to_shift[1]+1):], 
                                              empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                
        optimum_shift = np.unravel_index(np.argmax(scores,axis=None), scores.shape)
        optimum_breakpoint = breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shifts[optimum_shift[0]]] + \
                                            breakpoint_pair[(indices_to_shift[0]+1):indices_to_shift[1]] + \
                                            [breakpoint_pair[indices_to_shift[1]]+shifts[optimum_shift[1]]] + breakpoint_pair[(indices_to_shift[1]+1):]
        optimum_score = scores[optimum_shift]
        optimum_length = optimum_breakpoint[indices_of_array[1]] - optimum_breakpoint[indices_of_array[0]] + 4
    
    return optimum_breakpoint, optimum_score, optimum_length

def optimize_gene(gene, 
                  max_tile_size,
                  first_last_block_reduction,
                  block_size_range=block_size_range, 
                  slack=slack, 
                  empirical=bsaI_empirical, 
                  overhang_blacklist=overhang_blacklist): 
    
    #setup initial inputs to optimization
    gene_size = len(gene)
    protein_size = len(gene.translate())
        
    #exclude gene if it is too big
    if protein_size > 1000:
        print('Protein size too big!')
        
    #divide genes between 500 and 1000aa into two blocks
    elif protein_size > 620:
        print('Protein size too big! Will add two superblock (620aa+ proteins) soon.')
        
    else:
        #gene is one superblock 
        #print('Protein is one superblock.')
        block_size = block_size_range[0] + np.argmin(
            [abs((gene_size+2*first_last_block_reduction)/(i+block_size_range[0]) - \
                     round((gene_size+2*first_last_block_reduction)/(i+block_size_range[0]))) \
                 for i in range(0, block_size_range[1]-block_size_range[0])])
        
        
        # now, set initial breakpoints
        fragment_number = int((gene_size+2*first_last_block_reduction)/block_size)
        
        # if any of the tiles are too big?
        tile_lengths = [1000]
        while max(tile_lengths) > (max_tile_size-2*slack):
            fragment_number = fragment_number + 1
            first_breakpoint = 0
            last_breakpoint = gene_size-4
            step = (last_breakpoint - first_breakpoint + 2*first_last_block_reduction)/fragment_number
            evenly_spaced_floats = [first_breakpoint] + [step * i - first_last_block_reduction for i in range(1,fragment_number)] + [last_breakpoint]        
            initial_breakpoints = [[first_breakpoint,int(evenly_spaced_floats[1])+slack+2,last_breakpoint]] + \
                                    [[0,int(evenly_spaced_floats[i])-2-slack,int(evenly_spaced_floats[i+1])+slack+2,last_breakpoint] for i in range(1,len(evenly_spaced_floats)-2)] + \
                                    [[0,int(evenly_spaced_floats[-2])-2-slack,last_breakpoint]]
            tile_lengths = [int(evenly_spaced_floats[1])+slack+2-first_breakpoint+first_last_block_reduction+4] + \
                                    [int(evenly_spaced_floats[i+1])-int(evenly_spaced_floats[i])+2*(slack+2)+4 for i in range(1,len(evenly_spaced_floats)-2)] + \
                                    [last_breakpoint+4-int(evenly_spaced_floats[-2])+slack+2+first_last_block_reduction]
            

        #optimize each breakpoint
        optimum_breakpoints = []
        optimum_scores = []
        optimum_lengths = []
        oligo_array_indices = []
        for k,breakpoint in enumerate(initial_breakpoints):
            if len(breakpoint) == 3:
                indices_of_array = [0, 1] if k==0 else [1, 2]
                optimum_breakpoint, optimum_score, optimum_length = optimize_breakpoints(gene, breakpoint, [1], indices_of_array,
                                                            slack, empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                optimum_breakpoints.append(optimum_breakpoint)
                optimum_scores.append(optimum_score)
                optimum_lengths.append(optimum_length)
                oligo_array_indices.append(indices_of_array)
            else:
                indices_of_array = [1, 2]
                optimum_breakpoint, optimum_score, optimum_length = optimize_breakpoints(gene, breakpoint, [1, 2], indices_of_array,
                                                            slack, empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                optimum_breakpoints.append(optimum_breakpoint)
                optimum_scores.append(optimum_score)
                optimum_lengths.append(optimum_length)
                oligo_array_indices.append(indices_of_array)
    
    optimum_overlaps = [[str(gene[t:(t+4)]) for t in s] for s in optimum_breakpoints]
    if all([s >= 0.95 for s in optimum_scores]):
        print('All regions are high fidelity!')
    elif all([s >= 0.9 for s in optimum_scores]):
        print('Some regions are medium fidelity.')
    else:
        print('Some regions are low fidelity. Look closer')
        
    return optimum_breakpoints, optimum_overlaps, optimum_scores, optimum_lengths, oligo_array_indices

def generate_primer(DNA_seq,
                     Fwd=True,
                     extendtoCG=False,
                     smallest_primer_size=16,
                     largest_primer_size=30,
                     Tm=55):
    
    #Setup melting temperature arrays
    melt_temp_array = np.zeros(largest_primer_size-smallest_primer_size+1)
    
    if Fwd:
        DNA_seq_touse = DNA_seq
    else:
        DNA_seq_touse = DNA_seq.reverse_complement()
            
    #Make melting temperature arrays
    primer_length = 0
    for i in range(smallest_primer_size,largest_primer_size+1):
        melt_temp_array[i-smallest_primer_size] = mt.Tm_NN(DNA_seq_touse[0:i])
        
        #Pick F primer when Tm is first >F_Tm
        if (melt_temp_array[i-smallest_primer_size] >= Tm) & (primer_length==0):
            primer_length = i
    
    #If Tm isnt high enough after max bases, just set primer length to be max and hope it works
    if (primer_length == 0):
        primer_length = largest_primer_size
        
    if extendtoCG:
        while ((DNA_seq_touse[primer_length-1] == 'A') | (DNA_seq_touse[primer_length-1] == 'T')) & \
                    (primer_length < largest_primer_size):
            primer_length += 1
    
    return DNA_seq_touse[0:primer_length]

def make_mutations(region_name,
                       region,
                       region_flanks=[Seq(''),Seq('')],
                       nt_start=0, #zero-indexed!
                       wt_only=False,
                       synonymous=True,
                       stops='TAA',
                       all3ntdeletions=True,
                       mutation_list=False,
                       codons_ranked_by_usage=codons_ranked_by_usage,
                       aa_start=0,
                       aa_stop=None) :
                       
    oligo_array = {}
    #Check that region has size divisible by three
    if (len(region)/3 != len(region)//3) | (nt_start/3 != nt_start//3):
        print('Region is not translatable!')
        
    else:
        #add wt seq to oligo array
        oligo_name = region_name + '_WT'
        wt_seq = \
            region_flanks[0] + region + region_flanks[1]
        oligo_array[oligo_name] = wt_seq
        
        if not wt_only:
            
            # see if a mutation list was given
            if mutation_list == False:
                    
                #loop over amino acids
                for j in range(0,len(region),3):

                    #add all missense variants
                    aa = region[j:(j+3)].translate()
                    for aa_to in codons_ranked_by_usage.keys():
                        if aa_to != aa:
                            oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + str(aa_to)
                            seq_to_append = \
                                region_flanks[0] + \
                                region[0:j] + Seq(codons_ranked_by_usage[aa_to][0]) + \
                                region[(j+3):] + \
                                region_flanks[1]
                            oligo_array[oligo_name] = seq_to_append

                    #add synonymous variant if True and if possible, 
                    # using the most common codon that is NOT the codon in the gene
                    if synonymous:
                        if len(codons_ranked_by_usage[aa]) > 1:
                            oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + str(aa)
                            possible_codons = codons_ranked_by_usage[aa].copy()
                            possible_codons.remove(region[j:(j+3)])
                            seq_to_append = \
                                region_flanks[0] + \
                                region[0:j] + Seq(possible_codons[0]) + \
                                region[(j+3):] + \
                                region_flanks[1]
                            oligo_array[oligo_name] = seq_to_append

                    #add stops if true
                    if stops:
                        oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + 'X'
                        seq_to_append = \
                            region_flanks[0] + \
                            region[0:j] + Seq(stops) + \
                            region[(j+3):] + \
                            region_flanks[1]
                        oligo_array[oligo_name] = seq_to_append

                    #add all 3nt deletions if True
                    if all3ntdeletions:
                        for k in range(0,3):
                            if j+k+3 <= len(region):
                                oligo_name = region_name + '_' + 'del' + str(nt_start+j+k+1+3*aa_start)
                                seq_to_append = \
                                    region_flanks[0] + \
                                    region[0:(j+k)] + \
                                    region[(j+k+3):] + \
                                    region_flanks[1]
                                oligo_array[oligo_name] = seq_to_append
                                
            else:
                
                #loop over mutation list
                for i in range(len(mutation_list)):
                        
                    #iterate over every single aa change
                    oligo_name = region_name + '_' + 'variant' + str(i+1)
                    seq_to_append = region
                    for k,v in enumerate(mutation_list[i]):
                        aa_from = v[0]
                        aa_to = v[-1]
                        pos = int(v[1:-1])
                        j=3*(pos-(nt_start//3+1))
                        aa = region[j:(j+3)].translate()
                        if aa != aa_from:
                            print('Check mutation list!')
                        else:
                            seq_to_append = \
                                seq_to_append[0:j] + \
                                Seq(codons_ranked_by_usage[aa_to][0]) + \
                                seq_to_append[(j+3):]

                    # append oligo to array
                    seq_to_append = region_flanks[0] + \
                                    seq_to_append + \
                                    region_flanks[1]
                    oligo_array[oligo_name] = seq_to_append
        
    return oligo_array


def write_oligo_library(genes,
                        oligo_file='./oligo_test.csv',
                        primer_file='./primer_test.tsv',
                        gbl_file='./gbl_test.tsv',
                        gbl_large_file='./gbl_test_large.tsv',
                        amp_primer_key_file='./amp_primer_key.tsv',
                        breakpoint_file='./breakpoints.tsv',
                        primer_set=orthogonal_primers_touse,
                        codons_ranked_by_usage=codons_ranked_by_usage,
                        block_size_range=block_size_range, 
                        max_oligo_size=max_oligo_size,
                        slack=slack, 
                        empirical=bsaI_empirical, 
                        overhang_blacklist=overhang_blacklist,
                        validated_primer_set=False,
                        aa_start=False,
                        aa_stop=False,
                        wt_only=False,
                        synonymous=True,
                        stops='TAA',
                        all3ntdeletions=True,
                        mutations_to_use=False,
                        find_pcr_primers=True,
                        smallest_primer_size=16,
                        largest_primer_size=30,
                        Tm=55,
                        extendtoCG=True,
                        bsaI_firstoverlap='ACGA',
                        bsaI_lastoverlap='AATG',
                        all_blocks=True,
                        blocks_to_include=False,
                        tile_boundaries=False,
                        paqcIcapF=True,
                        paqcIcapR=True,
                        check_all_primers=True,
                        qc_melt_temp_threshold=32,
                        gblock_min_size=300,
                        gblock_large_threshold=1000,
                        randomsequencepad=randomsequencepad):
    
    #Split up primer set into F and R primers, cannot do more than 82 sublibraries
    oligo_primer_counter = 0
    oligo_array = {}
    amp_primers = {}
    gblocks = {}
    num_blocks = {}
    amp_primer_dict = {}
    breakpoint_dict = {}
    
    #Convert genes to Seq and genes to list
    gene_names = list(genes.keys())
    genes = [Seq(genes[gene_name].upper()) for gene_name in gene_names]
    
    #PaqCI and BsaI site sequences
    paqcI_seq = Seq('CACCTGC')
    paqcI_seqplusfour = Seq('CACCTGCCTAG')
    bsaI_seq = Seq('GGTCTC')
    bsaI_seqplusone = Seq('GGTCTCT')
    pcr_capseq = Seq('GGCTAC') + bsaI_seqplusone
    gbl_capseq_F = Seq('CCGCGTGATTACGAGTCG') + pcr_capseq
    gbl_capseq_R = Seq('GGGTTAGCAAGTGGCAGCCT') + pcr_capseq
    
    # set max size of a tile
    primer_len = len(primer_set['Forward Primer'][0])
    max_tile_size = max_oligo_size - 2*primer_len - 2*len(bsaI_seqplusone)
    first_last_block_reduction = len(paqcI_seqplusfour)
    
    # iterate over genes
    for r,gene in enumerate(genes):
        
        gene_name = gene_names[r]
        print('Processing gene ' + str(r+1) + ': ' + gene_name)
        
        # amino acid to start at
        if aa_start and gene_name in aa_start:
            aa_start_gene = aa_start[gene_name] - 1
        else:
            aa_start_gene = 0

        if aa_stop and gene_name in aa_stop:
            aa_stop_gene = aa_stop[gene_name]
        else:
            aa_stop_gene = None
        
        #exclude if gene size is not divisible by three
        if len(gene)/3 != len(gene)//3:
            print('Gene length is not divisible by 3!')
    
        #exclude if there is a paqcI site in the gene
        elif any([True for kmer in build_kmers(gene, len(paqcI_seq)) if kmer==paqcI_seq]) | \
            any([True for kmer in build_kmers(gene.reverse_complement(), len(paqcI_seq)) if kmer==paqcI_seq]):
            print('Gene has PaqCI site!')
        
        #exclude if there is a BsaI site in the gene
        elif any([True for kmer in build_kmers(gene, len(bsaI_seq)) if kmer==bsaI_seq]) | \
            any([True for kmer in build_kmers(gene.reverse_complement(), len(bsaI_seq)) if kmer==bsaI_seq]):
            print('Gene has BsaI site!')
            
        else:
            print('Gene has no PaqCI or BsaI sites! Performing GoldenGate optimization...')
            
            #cap gene with BsaI breakpoints and possible PaqCI sites 
            if paqcIcapF & paqcIcapR:
                gene_capped = bsaI_firstoverlap + paqcI_seqplusfour + \
                        gene + paqcI_seqplusfour.reverse_complement() + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap + paqcI_seqplusfour)
                capping_length_R = len(paqcI_seqplusfour.reverse_complement() + bsaI_lastoverlap)
            elif paqcIcapF:
                gene_capped = bsaI_firstoverlap + paqcI_seqplusfour + \
                        gene + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap + paqcI_seqplusfour)
                capping_length_R = len(bsaI_lastoverlap)
            elif paqcIcapR:
                gene_capped = bsaI_firstoverlap + \
                        gene + paqcI_seqplusfour.reverse_complement() + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap)
                capping_length_R = len(paqcI_seqplusfour.reverse_complement() + bsaI_lastoverlap)
            else:
                gene_capped = bsaI_firstoverlap + gene + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap)
                capping_length_R = len(bsaI_lastoverlap)
            
            #Optimize gene if tile boundaries are not given
            if tile_boundaries is False:
                optimum_breakpoints, optimum_overlaps, optimum_scores, optimum_lengths, oligo_array_indices = \
                optimize_gene(gene_capped, 
                          max_tile_size=max_tile_size,
                          first_last_block_reduction=first_last_block_reduction,
                          block_size_range=block_size_range, 
                          slack=slack, 
                          empirical=bsaI_empirical, 
                          overhang_blacklist=overhang_blacklist)
                pprint.pprint({'Optimum Breakpoints': optimum_breakpoints, 
                       'Optimum Overlaps': optimum_overlaps, 
                       'Optimum Scores': optimum_scores})
                num_blocks[gene_name] = len(optimum_breakpoints)
            else:
                optimum_breakpoints = tile_boundaries[gene_name]
                if len(optimum_breakpoints[0])==3:
                    if len(optimum_breakpoints)>1:
                        #multiple tiles, including one at beginning of gene
                        oligo_array_indices = [[0,1]] + [[1,2]]*(len(optimum_breakpoints)-1)
                    else:
                        # one tile
                        if ((optimum_breakpoints[1]-optimum_breakpoints[0]) > (optimum_breakpoints[2]-optimum_breakpoints[1])):
                            # at end of gene
                            oligo_array_indices = [[1,2]]
                        else:
                            # at beginning of gene
                            oligo_array_indices = [[0,1]]
                else:
                    # multiple tiles, starting in the middle
                    oligo_array_indices = [[1,2]]*(len(optimum_breakpoints))
                num_blocks[gene_name] = len(optimum_breakpoints)
                
            
            #add primers for gene_F and gene_R that are repeated constantly throughout the PCRs
            #note: should probably prevalidate these primers!
            if find_pcr_primers:
                F_primer = generate_primer(gene,
                                           Fwd=True,
                                           extendtoCG=extendtoCG,
                                           smallest_primer_size=smallest_primer_size,
                                           largest_primer_size=largest_primer_size,
                                           Tm=Tm)
                F_primer = pcr_capseq + bsaI_firstoverlap + paqcI_seqplusfour + F_primer
                amp_primers[gene_name+'_gene'+'_ampF'] = F_primer
                R_primer = generate_primer(gene,
                                           Fwd=False,
                                           extendtoCG=extendtoCG,
                                           smallest_primer_size=smallest_primer_size,
                                           largest_primer_size=largest_primer_size,
                                           Tm=Tm)
                R_primer = pcr_capseq + Seq(bsaI_lastoverlap).reverse_complement() + \
                            paqcI_seqplusfour + R_primer
                amp_primers[gene_name+'_gene'+'_ampR'] = R_primer
            
            #make oligos, primers, gblocks for each block
            for i,breakpoint in enumerate(optimum_breakpoints):
                
                #find indices of breakpoint that correspond to oligo vs need to be PCRed/gblock
                pcr_indices = [[j,j+1] for j in range(len(breakpoint)-1)]
                pcr_indices.remove(oligo_array_indices[i])
                
                #find mutagenic window of oligo
                oligo_breaks = [breakpoint[j] for j in oligo_array_indices[i]]
                oligo_mutagenic_window = [int(3*np.ceil(max(oligo_breaks[0]+4-capping_length_F,3)/3)), int(3*np.floor(min(oligo_breaks[1]-capping_length_F,len(gene)-1)/3))]
                
                #subset the right block if needed
                if (all_blocks == True) | ((i+1) in blocks_to_include[r] if blocks_to_include != False else True): #subset on allowed blocks
                
                    #add pcr primers and gblocks, one segment at a time
                    for k,pcr_index in enumerate(pcr_indices):
                        piece_name = gene_name + '_block' + str(i+1) + '_s' + str(k+1)
                        pcr_breaks = [breakpoint[j] for j in pcr_index]
                                            
                        #get pcr primers
                        if find_pcr_primers:
                            if pcr_breaks[0] == breakpoint[0]: #Fragment beginning at gene start 
                                R_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=False,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                R_primer = pcr_capseq + R_primer
                                amp_primers[piece_name+'_ampR'] = R_primer
                            elif pcr_breaks[1] == breakpoint[-1]: #Fragment ending at gene end
                                F_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=True,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                F_primer = pcr_capseq + F_primer
                                amp_primers[piece_name+'_ampF'] = F_primer
                            else:
                                F_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=True,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                F_primer = pcr_capseq + F_primer
                                R_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=False,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                R_primer = pcr_capseq + R_primer
                                amp_primers[piece_name+'_ampF'] = F_primer
                                amp_primers[piece_name+'_ampR'] = R_primer

                        #make gblocks
                        gbl = gbl_capseq_F + gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)] + gbl_capseq_R.reverse_complement()
                        gblocks[piece_name] = gbl
                        
                    #add oligos to oligo array, checking first for validated primers if they are given
                    validated=False
                    if validated_primer_set is not False:
                        validated_combo = validated_primer_set.query('Gene == @gene_name & Block == (@i+1)')
                        if not validated_combo.empty:
                            validated=True
                            name_primer_F, primer_F, name_primer_R, primer_R = \
                                validated_combo[['Forward Name',
                                                  'Forward Primer',
                                                  'Reverse Name',
                                                  'Reverse Primer']].values[0]
                        else:
                            name_primer_F, primer_F, name_primer_R, primer_R = \
                                primer_set.iloc[oligo_primer_counter,][['Forward Name',
                                                                      'Forward Primer',
                                                                      'Reverse Name',
                                                                      'Reverse Primer']]
                            oligo_primer_counter += 1
                    else:
                        name_primer_F, primer_F, name_primer_R, primer_R = \
                                primer_set.iloc[oligo_primer_counter,][['Forward Name',
                                                                      'Forward Primer',
                                                                      'Reverse Name',
                                                                      'Reverse Primer']]
                        oligo_primer_counter += 1
                        
                    # if mutations are given, use those - otherwise make all mutations or wtonly
                    if mutations_to_use == False:
                        add_on_array = make_mutations(gene_name + '_block' + str(i+1),
                                           gene[oligo_mutagenic_window[0]:oligo_mutagenic_window[1]],
                                           region_flanks=[Seq(primer_F) + \
                                                          bsaI_seqplusone + \
                                                          gene_capped[oligo_breaks[0]:(oligo_mutagenic_window[0]+capping_length_F)] ,
                                                          gene_capped[(oligo_mutagenic_window[1]+capping_length_F):(oligo_breaks[1]+4)] + \
                                                          bsaI_seqplusone.reverse_complement() + \
                                                          Seq(primer_R).reverse_complement()],
                                           nt_start=oligo_mutagenic_window[0],
                                           wt_only=wt_only,
                                           synonymous=synonymous,
                                           stops=stops,
                                           all3ntdeletions=all3ntdeletions,
                                           codons_ranked_by_usage=codons_ranked_by_usage,
                                           aa_start=aa_start_gene)
                    else:
                        add_on_array = make_mutations(gene_name + '_block' + str(i+1),
                                           gene[oligo_mutagenic_window[0]:oligo_mutagenic_window[1]],
                                           region_flanks=[Seq(primer_F) + \
                                                          bsaI_seqplusone + \
                                                          gene_capped[oligo_breaks[0]:(oligo_mutagenic_window[0]+capping_length_F)] ,
                                                          gene_capped[(oligo_mutagenic_window[1]+capping_length_F):(oligo_breaks[1]+4)] + \
                                                          bsaI_seqplusone.reverse_complement() + \
                                                          Seq(primer_R).reverse_complement()],
                                           nt_start=oligo_mutagenic_window[0],
                                           mutation_list=mutations_to_use[(gene_name,i+1)],
                                           codons_ranked_by_usage=codons_ranked_by_usage,
                                           aa_start=aa_start_gene,
                                           aa_stop=aa_stop_gene)
                    oligo_array.update(add_on_array)
                    amp_primer_dict.update({(gene_name,i+1): (name_primer_F,primer_F,name_primer_R,primer_R,validated)})
                    breakpoint_dict.update({(gene_name,i+1): oligo_mutagenic_window})
                    
    #Check that max oligo is less than the max oligo length
    if sum([len(s)>max_oligo_size for s in oligo_array.values()]) == 0:
        print('All oligos are below the maximum 250bp!')
    else:
        print('Some oligos are TOO BIG!')
        
    #Check for nonspecific amplification
    wt_oligos = {tuple([key.split('_')[0],
                        int((key.split('_block')[1]).split('_')[0])]
                      ):oligo_array[key] \
                     for key in oligo_array.keys() if 'WT' in key}
    nonspecific_primers = post_qc(amp_primer_dict, 
                                  wt_oligos,
                                  primer_set, 
                                  melt_temp_threshold=qc_melt_temp_threshold,
                                  check_all_primers=check_all_primers)
    
    # Check that all mutagenic windows overlap
    breakpoint_df = pd.DataFrame.from_dict(breakpoint_dict, orient='index', columns=['Mutagenesis Start','Mutagenesis End'])
    breakpoint_df.index = pd.MultiIndex.from_tuples(breakpoint_dict.keys())
    breakpoint_df = breakpoint_df.reset_index().rename(columns={'level_0':'Gene',
                                                                'level_1':'Block'})
    if (mutations_to_use == False) and (all_blocks == True):
        missed_counter = 0
        for r,gene_group_breakpoints in breakpoint_df.groupby('Gene'):
            for k,row in gene_group_breakpoints.iterrows():
                # look at whether the current row start is later than the last row end
                if row['Block'] > 1:
                    if row['Mutagenesis Start'] > end:
                        missed_counter += 1
                        print('Mutagenic window missed at ' + str(r) + ' block ' + str(row['Block']))
                start,end = row['Mutagenesis Start'],row['Mutagenesis End']
        if missed_counter == 0:
            print('All mutagenic windows overlap!')
        else:
            print(str(missed_counter) + ' number of times the mutagenic window does not close!')
                
    #Remove any oligos with additional BsaI sites or PaqCI sites
    bad_oligos = []
    for name,oligo in oligo_array.items():
        #check for PaqCI sites
        paqcI_F = sum([True for kmer in build_kmers(oligo, len(paqcI_seq)) if kmer==paqcI_seq])
        paqcI_R = sum([True for kmer in build_kmers(oligo.reverse_complement(), len(paqcI_seq)) if kmer==paqcI_seq])
        #check that oligo is block 1 if it contains a paqcI site in the forward orientation 
        if paqcI_F > 0:
            if ('block1' not in name) | (paqcI_F > 1):
                bad_oligos.append(name)
        #check that the oligo is block final if it contains a paqcI site in the reverse orientation
        if paqcI_R > 0:
            if ('block'+str(num_blocks[name.split('_')[0]]) not in name) | (paqcI_R > 1):
                bad_oligos.append(name)
        #check for more than one BsaI site
        bsaI_F = sum([True for kmer in build_kmers(oligo, len(bsaI_seq)) if kmer==bsaI_seq])
        bsaI_R = sum([True for kmer in build_kmers(oligo.reverse_complement(), len(bsaI_seq)) if kmer==bsaI_seq])
        if (bsaI_F != 1) | (bsaI_R != 1):
            bad_oligos.append(name)
    bad_oligos=np.unique(bad_oligos)
    for oligo_name in bad_oligos:
        del oligo_array[oligo_name]
    print(str(len(bad_oligos)) + ' oligos deleted due to errant restriction sites.')
    
    #Remove any duplicate oligos
    new_dict = {}
    seen_values = set()
    counter=0
    for key, value in oligo_array.items():
        if value not in seen_values:
            new_dict[key] = value
            seen_values.add(value)
        else:
            counter += 1
    print(str(counter) + ' oligos removed due to duplication.')
    oligo_array = new_dict
    del new_dict
    
    #write oligo array to file
    with open(oligo_file, 'w') as f:
        for key in oligo_array.keys():
            f.write("%s,%s\n"%(key,oligo_array[key]))
    f.close()
            
    #write primers to file
    if find_pcr_primers:
        primer_order_sheet = []
        for key in amp_primers.keys():
            primer_order_sheet.append(key + '\t' + \
                     str(amp_primers[key]) + \
                     '\t' + '25nm' + '\t' + 'STD\n')
        print(*primer_order_sheet)
        with open(primer_file, 'w') as f:
            for line in primer_order_sheet:
                f.write(line)
        f.close()
    
    #write amplification primer key to file
    amp_primer_key = ['Gene' + '\t' + 'Block' + '\t' + \
                      'Forward Primer Well' + '\t' + 'Forward Primer' + '\t' + \
                      'Reverse Primer Well' + '\t' + 'Reverse Primer' + '\t' + 'Validated' + '\n']
    for key in amp_primer_dict.keys():
        genename, geneblock = key[0], str(key[1])
        name_primer_F, primer_F, name_primer_R, primer_R, validated = amp_primer_dict[key]
        amp_primer_key.append(genename + '\t' + geneblock + '\t' + \
                 name_primer_F + '\t' + primer_F + '\t' + \
                 name_primer_R + '\t' + primer_R + '\t' + str(validated) + '\n')
    print(*amp_primer_key)
    with open(amp_primer_key_file, 'w') as f:
        for line in amp_primer_key:
            f.write(line)
    f.close()
    
    #write breakpoint dict to file
    breakpoint_df.to_csv(breakpoint_file, sep='\t')
    
    #write gblocks to file
    gblock_order_sheet = []
    gblock_large_order_sheet = []
    for key in gblocks.keys():
        # pad gblock if it is not 300bp for Twist
        if len(gblocks[key]) < gblock_min_size:
            gblocks[key] = Seq(randomsequencepad[0:(gblock_min_size-len(gblocks[key]))]) + gblocks[key]
        if len(gblocks[key]) < gblock_large_threshold:
            gblock_order_sheet.append(key + '\t' + \
                     str(gblocks[key]) + '\n')
        else:
            gblock_large_order_sheet.append(key + '\t' + \
                     str(gblocks[key]) + '\n')
    print(*gblock_order_sheet)
    print(*gblock_large_order_sheet)
    with open(gbl_file, 'w') as f:
        for line in gblock_order_sheet:
            f.write(line)
    f.close()
    with open(gbl_large_file, 'w') as f:
        for line in gblock_large_order_sheet:
            f.write(line)
    f.close()
    
    return oligo_array,amp_primers,gblocks,amp_primer_dict,breakpoint_df
                

In [39]:
#redefine functions for cterm tagging (again) 20250613
def post_qc(amp_primer_set, wt_oligos, primer_set, melt_temp_threshold = 35, check_all_primers=True):
    print("Running QC for primer specificity on WT oligos")
    f_primer_map = {}
    r_primer_map = {}
    # invert the primer to subpool map
    for k, v in amp_primer_set.items():
        f_primer_map[v[1]] = f_primer_map.get(v[1], []) + [k]
        r_primer_map[v[3]] = r_primer_map.get(v[3], []) + [k]
    
    # initialize list of nonspecific problems
    nonspecific = {}
    
    # add unused primers if check_all_primers
    if check_all_primers:
        all_f_primers = np.unique(primer_set['Forward Primer'])
        all_r_primers = np.unique(primer_set['Reverse Primer'])
        for f_primer in all_f_primers:
            if f_primer not in f_primer_map.keys():
                f_primer_map[f_primer] = []
        for r_primer in all_r_primers:
            if r_primer not in r_primer_map.keys():
                r_primer_map[r_primer] = []
        
    for f_primer, subpools_used in f_primer_map.items():
    # iterate over every barcode primer pair and match to each oligo to check for nonspecific amplification
        anneal_locs = []
        for subpoolcheck, fragmentcheck in wt_oligos.items():  # iterate over every WT oligo
            if (subpoolcheck not in subpools_used):  # ignore designed annealing (same name)
                if check_nonspecific(f_primer, fragmentcheck, Tm_rem = melt_temp_threshold, verbose=False) > 0: #use high Tm_rem
                    anneal_locs.append(subpoolcheck)
        if anneal_locs:
            nonspecific.update({f_primer:[a[0] + '_block' + str(a[1]+1) for a in anneal_locs]})
    for r_primer, subpools_used in r_primer_map.items():
    # iterate over every barcode primer pair and match to each oligo to check for nonspecific amplification
        anneal_locs = []
        for subpoolcheck, fragmentcheck in wt_oligos.items():  # iterate over every WT oligo
            if (subpoolcheck not in subpools_used):  # ignore designed annealing (same name)
                if check_nonspecific(r_primer, fragmentcheck, Tm_rem = melt_temp_threshold, verbose=False) > 0: #use high Tm_rem
                    anneal_locs.append(subpoolcheck)
        if anneal_locs:
            nonspecific.update({r_primer:[a[0] + '_block' + str(a[1]+1) for a in anneal_locs]})
    if nonspecific:
        print("Nonspecific Primers: (Manually removing primer sequence recommended)")
        print(nonspecific)
    else:
        print("No non-specific primers detected")
        
    return nonspecific

def build_kmers(sequence, 
                ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)

    return kmers

def compute_overlaps(breakpoints, 
                     inclusion_array, 
                     gene):
    
    overlaps = [[gene[val:val+4].reverse_complement(), gene[val:val+4]] for val in breakpoints]
    counter = 0
    for val in inclusion_array:
        if val == -1:
            (overlaps[counter][1],overlaps[counter+1][0]) = (overlaps[counter+1][0],overlaps[counter][1])
            counter += 1
        elif val == 0:
            overlaps[counter][1] = overlaps[counter+1][1]
            del overlaps[counter+1]
        
    return overlaps

def score_breakpoints(gene, 
                      breakpoint_pair, 
                      empirical, 
                      overhang_blacklist=overhang_blacklist):
    
    #subset empirical matrix by the set of all overlaps
    all_overlaps = []
    for breakpoint in breakpoint_pair:
        all_overlaps.append(gene[breakpoint:(breakpoint+4)])
        all_overlaps.append(gene[breakpoint:(breakpoint+4)].reverse_complement())
    all_overlaps = [str(o) for o in all_overlaps]
    if (len(np.unique(all_overlaps)) == len(all_overlaps)) & (len(set(all_overlaps).intersection(set(overhang_blacklist))) == 0):
        empirical_subset = empirical.loc[all_overlaps,all_overlaps]

        #compute fidelity score
        empirical_subset = empirical_subset/empirical_subset.sum(axis=1)
        fidelity_score = 1
        for breakpoint in breakpoint_pair:
            fidelity_score = fidelity_score * \
                empirical_subset.loc[str(gene[breakpoint:(breakpoint+4)]),
                                     str(gene[breakpoint:(breakpoint+4)].reverse_complement())]
    
    else:
        fidelity_score = 0
    
    return fidelity_score
    
def optimize_breakpoints(gene, 
                         breakpoint_pair, 
                         indices_to_shift, 
                         indices_of_array,
                         slack, 
                         empirical=bsaI_empirical, 
                         overhang_blacklist=overhang_blacklist):
    
    #compute all enrichments
    shifts = list(range(-slack,slack+1))
    if (len(indices_to_shift) > 2) | (len(indices_to_shift) < 1):
        print('Error -- too many or too few breakpoints!')
        optimum_breakpoint = breakpoint_pair
        optimum_score = 0
    elif (len(indices_to_shift) == 1): #external pair
        scores = [0]*len(shifts)
        for i,shift in enumerate(shifts):
            scores[i] = score_breakpoints(gene, breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shift] + breakpoint_pair[(indices_to_shift[0]+1):], 
                                          empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
        
        optimum_shift = np.argmax(scores)
        optimum_breakpoint = breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shifts[optimum_shift]] + breakpoint_pair[(indices_to_shift[0]+1):]
        optimum_score = scores[optimum_shift]
        optimum_length = optimum_breakpoint[indices_of_array[1]] - optimum_breakpoint[indices_of_array[0]]
            
            
    else: #internal pair
        indices_to_shift = sorted(indices_to_shift)
        scores = np.zeros((len(shifts),len(shifts)))
        for i,shift1 in enumerate(shifts):
            for j,shift2 in enumerate(shifts):
                scores[i,j] = score_breakpoints(gene, breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shift1] + \
                                                            breakpoint_pair[(indices_to_shift[0]+1):indices_to_shift[1]] + \
                                                            [breakpoint_pair[indices_to_shift[1]]+shift2] + breakpoint_pair[(indices_to_shift[1]+1):], 
                                              empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                
        optimum_shift = np.unravel_index(np.argmax(scores,axis=None), scores.shape)
        optimum_breakpoint = breakpoint_pair[0:indices_to_shift[0]] + [breakpoint_pair[indices_to_shift[0]]+shifts[optimum_shift[0]]] + \
                                            breakpoint_pair[(indices_to_shift[0]+1):indices_to_shift[1]] + \
                                            [breakpoint_pair[indices_to_shift[1]]+shifts[optimum_shift[1]]] + breakpoint_pair[(indices_to_shift[1]+1):]
        optimum_score = scores[optimum_shift]
        optimum_length = optimum_breakpoint[indices_of_array[1]] - optimum_breakpoint[indices_of_array[0]] + 4
    
    return optimum_breakpoint, optimum_score, optimum_length

def optimize_gene(gene, 
                  max_tile_size,
                  first_last_block_reduction,
                  block_size_range=block_size_range, 
                  slack=slack, 
                  empirical=bsaI_empirical, 
                  overhang_blacklist=overhang_blacklist): 
    
    #setup initial inputs to optimization
    gene_size = len(gene)
    protein_size = len(gene.translate())
        
    #exclude gene if it is too big
    if protein_size > 1000:
        print('Protein size too big!')
        
    #divide genes between 500 and 1000aa into two blocks
    elif protein_size > 620:
        print('Protein size too big! Will add two superblock (620aa+ proteins) soon.')
        
    else:
        #gene is one superblock 
        #print('Protein is one superblock.')
        block_size = block_size_range[0] + np.argmin(
            [abs((gene_size+2*first_last_block_reduction)/(i+block_size_range[0]) - \
                     round((gene_size+2*first_last_block_reduction)/(i+block_size_range[0]))) \
                 for i in range(0, block_size_range[1]-block_size_range[0])])
        
        
        # now, set initial breakpoints
        fragment_number = int((gene_size+2*first_last_block_reduction)/block_size)
        
        # if any of the tiles are too big?
        tile_lengths = [1000]
        while max(tile_lengths) > (max_tile_size-2*slack):
            fragment_number = fragment_number + 1
            first_breakpoint = 0
            last_breakpoint = gene_size-4
            step = (last_breakpoint - first_breakpoint + 2*first_last_block_reduction)/fragment_number
            evenly_spaced_floats = [first_breakpoint] + [step * i - first_last_block_reduction for i in range(1,fragment_number)] + [last_breakpoint]        
            initial_breakpoints = [[first_breakpoint,int(evenly_spaced_floats[1])+slack+2,last_breakpoint]] + \
                                    [[0,int(evenly_spaced_floats[i])-2-slack,int(evenly_spaced_floats[i+1])+slack+2,last_breakpoint] for i in range(1,len(evenly_spaced_floats)-2)] + \
                                    [[0,int(evenly_spaced_floats[-2])-2-slack,last_breakpoint]]
            tile_lengths = [int(evenly_spaced_floats[1])+slack+2-first_breakpoint+first_last_block_reduction+4] + \
                                    [int(evenly_spaced_floats[i+1])-int(evenly_spaced_floats[i])+2*(slack+2)+4 for i in range(1,len(evenly_spaced_floats)-2)] + \
                                    [last_breakpoint+4-int(evenly_spaced_floats[-2])+slack+2+first_last_block_reduction]
            

        #optimize each breakpoint
        optimum_breakpoints = []
        optimum_scores = []
        optimum_lengths = []
        oligo_array_indices = []
        for k,breakpoint in enumerate(initial_breakpoints):
            if len(breakpoint) == 3:
                indices_of_array = [0, 1] if k==0 else [1, 2]
                optimum_breakpoint, optimum_score, optimum_length = optimize_breakpoints(gene, breakpoint, [1], indices_of_array,
                                                            slack, empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                optimum_breakpoints.append(optimum_breakpoint)
                optimum_scores.append(optimum_score)
                optimum_lengths.append(optimum_length)
                oligo_array_indices.append(indices_of_array)
            else:
                indices_of_array = [1, 2]
                optimum_breakpoint, optimum_score, optimum_length = optimize_breakpoints(gene, breakpoint, [1, 2], indices_of_array,
                                                            slack, empirical=bsaI_empirical, overhang_blacklist=overhang_blacklist)
                optimum_breakpoints.append(optimum_breakpoint)
                optimum_scores.append(optimum_score)
                optimum_lengths.append(optimum_length)
                oligo_array_indices.append(indices_of_array)
    
    optimum_overlaps = [[str(gene[t:(t+4)]) for t in s] for s in optimum_breakpoints]
    if all([s >= 0.95 for s in optimum_scores]):
        print('All regions are high fidelity!')
    elif all([s >= 0.9 for s in optimum_scores]):
        print('Some regions are medium fidelity.')
    else:
        print('Some regions are low fidelity. Look closer')
        
    return optimum_breakpoints, optimum_overlaps, optimum_scores, optimum_lengths, oligo_array_indices

def generate_primer(DNA_seq,
                     Fwd=True,
                     extendtoCG=False,
                     smallest_primer_size=16,
                     largest_primer_size=30,
                     Tm=55):
    
    #Setup melting temperature arrays
    melt_temp_array = np.zeros(largest_primer_size-smallest_primer_size+1)
    
    if Fwd:
        DNA_seq_touse = DNA_seq
    else:
        DNA_seq_touse = DNA_seq.reverse_complement()
            
    #Make melting temperature arrays
    primer_length = 0
    for i in range(smallest_primer_size,largest_primer_size+1):
        melt_temp_array[i-smallest_primer_size] = mt.Tm_NN(DNA_seq_touse[0:i])
        
        #Pick F primer when Tm is first >F_Tm
        if (melt_temp_array[i-smallest_primer_size] >= Tm) & (primer_length==0):
            primer_length = i
    
    #If Tm isnt high enough after max bases, just set primer length to be max and hope it works
    if (primer_length == 0):
        primer_length = largest_primer_size
        
    if extendtoCG:
        while ((DNA_seq_touse[primer_length-1] == 'A') | (DNA_seq_touse[primer_length-1] == 'T')) & \
                    (primer_length < largest_primer_size):
            primer_length += 1
    
    return DNA_seq_touse[0:primer_length]

def make_mutations(region_name,
                       region,
                       region_flanks=[Seq(''),Seq('')],
                       nt_start=0, #zero-indexed!
                       wt_only=False,
                       synonymous=True,
                       stops='TAA',
                       all3ntdeletions=True,
                       mutation_list=False,
                       codons_ranked_by_usage=codons_ranked_by_usage,
                       aa_start=0):
                       
    oligo_array = {}
    #Check that region has size divisible by three
    if (len(region)/3 != len(region)//3) | (nt_start/3 != nt_start//3):
        print('Region is not translatable!')
        
    else:
        #add wt seq to oligo array
        oligo_name = region_name + '_WT'
        wt_seq = \
            region_flanks[0] + region + region_flanks[1]
        oligo_array[oligo_name] = wt_seq
        
        if not wt_only:
            
            # see if a mutation list was given
            if mutation_list == False:
                    
                #loop over amino acids
                for j in range(0,len(region),3):

                    #add all missense variants
                    aa = region[j:(j+3)].translate()
                    for aa_to in codons_ranked_by_usage.keys():
                        if aa_to != aa:
                            oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + str(aa_to)
                            seq_to_append = \
                                region_flanks[0] + \
                                region[0:j] + Seq(codons_ranked_by_usage[aa_to][0]) + \
                                region[(j+3):] + \
                                region_flanks[1]
                            oligo_array[oligo_name] = seq_to_append

                    #add synonymous variant if True and if possible, 
                    # using the most common codon that is NOT the codon in the gene
                    if synonymous:
                        if len(codons_ranked_by_usage[aa]) > 1:
                            oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + str(aa)
                            possible_codons = codons_ranked_by_usage[aa].copy()
                            possible_codons.remove(region[j:(j+3)])
                            seq_to_append = \
                                region_flanks[0] + \
                                region[0:j] + Seq(possible_codons[0]) + \
                                region[(j+3):] + \
                                region_flanks[1]
                            oligo_array[oligo_name] = seq_to_append

                    #add stops if true
                    if stops:
                        oligo_name = region_name + '_' + str(aa) + str((nt_start+j)//3+1+aa_start) + 'X'
                        seq_to_append = \
                            region_flanks[0] + \
                            region[0:j] + Seq(stops) + \
                            region[(j+3):] + \
                            region_flanks[1]
                        oligo_array[oligo_name] = seq_to_append

                    #add all 3nt deletions if True
                    if all3ntdeletions:
                        for k in range(0,3):
                            if j+k+3 <= len(region):
                                oligo_name = region_name + '_' + 'del' + str(nt_start+j+k+1+3*aa_start)
                                seq_to_append = \
                                    region_flanks[0] + \
                                    region[0:(j+k)] + \
                                    region[(j+k+3):] + \
                                    region_flanks[1]
                                oligo_array[oligo_name] = seq_to_append
                                
            else:
                
                #loop over mutation list
                for i in range(len(mutation_list)):
                        
                    #iterate over every single aa change
                    oligo_name = region_name + '_' + 'variant' + str(i+1)
                    seq_to_append = region
                    for k,v in enumerate(mutation_list[i]):
                        aa_from = v[0]
                        aa_to = v[-1]
                        pos = int(v[1:-1])
                        j=3*(pos-(nt_start//3+1))
                        aa = region[j:(j+3)].translate()
                        if aa != aa_from:
                            print('Check mutation list!')
                        else:
                            seq_to_append = \
                                seq_to_append[0:j] + \
                                Seq(codons_ranked_by_usage[aa_to][0]) + \
                                seq_to_append[(j+3):]

                    # append oligo to array
                    seq_to_append = region_flanks[0] + \
                                    seq_to_append + \
                                    region_flanks[1]
                    oligo_array[oligo_name] = seq_to_append
        
    return oligo_array


def write_oligo_library(genes,
                        oligo_file='./oligo_test.csv',
                        primer_file='./primer_test.tsv',
                        gbl_file='./gbl_test.tsv',
                        gbl_large_file='./gbl_test_large.tsv',
                        amp_primer_key_file='./amp_primer_key.tsv',
                        breakpoint_file='./breakpoints.tsv',
                        primer_set=orthogonal_primers_touse,
                        codons_ranked_by_usage=codons_ranked_by_usage,
                        block_size_range=block_size_range, 
                        max_oligo_size=max_oligo_size,
                        slack=slack, 
                        empirical=bsaI_empirical, 
                        overhang_blacklist=overhang_blacklist,
                        validated_primer_set=False,
                        aa_start=False,
                        wt_only=False,
                        synonymous=True,
                        stops='TAA',
                        all3ntdeletions=True,
                        mutations_to_use=False,
                        find_pcr_primers=True,
                        smallest_primer_size=16,
                        largest_primer_size=30,
                        Tm=55,
                        extendtoCG=True,
                        bsaI_firstoverlap='CGTC',
                        bsaI_lastoverlap='GCAT',
                        all_blocks=True,
                        blocks_to_include=False,
                        tile_boundaries=False,
                        paqcIcapF=True,
                        paqcIcapR=True,
                        check_all_primers=True,
                        qc_melt_temp_threshold=32,
                        gblock_min_size=300,
                        gblock_large_threshold=1000,
                        randomsequencepad=randomsequencepad):
    
    #Split up primer set into F and R primers, cannot do more than 82 sublibraries
    oligo_primer_counter = 0
    oligo_array = {}
    amp_primers = {}
    gblocks = {}
    num_blocks = {}
    amp_primer_dict = {}
    breakpoint_dict = {}
    
    #Convert genes to Seq and genes to list
    gene_names = list(genes.keys())
    genes = [Seq(genes[gene_name].upper()) for gene_name in gene_names]
    
    #PaqCI and BsaI site sequences and overhangs for paqcI
    paqcI_seq = Seq('CACCTGC')
    paqcI_overhang_nterm_ctag = Seq('CCACC')
    paqcI_overhang_cterm_ctag = Seq('ACGGG')
    paqcI_overhang_nterm_ntag = Seq('TGGC')
    paqcI_overhang_cterm_ntag = Seq('TAGG')
    paqcI_seqplusfour = Seq('CACCTGCCTAG')
    bsaI_seq = Seq('GGTCTC')
    bsaI_seqplusone = Seq('GGTCTCT')
    pcr_capseq = Seq('GGCTAC') + bsaI_seqplusone
    gbl_capseq_F = Seq('CCGCGTGATTACGAGTCG') + pcr_capseq
    gbl_capseq_R = Seq('GGGTTAGCAAGTGGCAGCCT') + pcr_capseq
    
    # set max size of a tile
    primer_len = len(primer_set['Forward Primer'][0])
    max_tile_size = max_oligo_size - 2*primer_len - 2*len(bsaI_seqplusone)
    first_last_block_reduction = len(paqcI_seqplusfour)
    
    # iterate over genes
    for r,gene in enumerate(genes):
        
        gene_name = gene_names[r]
        print('Processing gene ' + str(r+1) + ': ' + gene_name)
        
        # amino acid to start at
        if aa_start != False:
            if gene_name in aa_start.keys():
                aa_start_gene=aa_start[gene_name]-1
            else:
                aa_start_gene=0
        else:
            aa_start_gene=0
        
        #exclude if gene size is not divisible by three
        if len(gene)/3 != len(gene)//3:
            print('Gene length is not divisible by 3!')
    
        #exclude if there is a paqcI site in the gene
        elif any([True for kmer in build_kmers(gene, len(paqcI_seq)) if kmer==paqcI_seq]) | \
            any([True for kmer in build_kmers(gene.reverse_complement(), len(paqcI_seq)) if kmer==paqcI_seq]):
            print('Gene has paqcI site!')
        
        #exclude if there is a BsaI site in the gene
        elif any([True for kmer in build_kmers(gene, len(bsaI_seq)) if kmer==bsaI_seq]) | \
            any([True for kmer in build_kmers(gene.reverse_complement(), len(bsaI_seq)) if kmer==bsaI_seq]):
            print('Gene has BsaI site!')
            
        else:
            print('Gene has no paqcI or BsaI sites! Performing GoldenGate optimization...')
            
            #cap gene with BsaI breakpoints and possible paqcI sites 
            if paqcIcapF & paqcIcapR:
                gene_capped = bsaI_firstoverlap + paqcI_seqplusfour +  paqcI_overhang_nterm_ctag  +\
                        gene + paqcI_overhang_cterm_ctag + paqcI_seqplusfour.reverse_complement() + \
                        bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap + paqcI_seqplusfour + paqcI_overhang_nterm_ctag)
                capping_length_R = len(paqcI_overhang_cterm_ctag + paqcI_seqplusfour.reverse_complement() + \
                                   bsaI_lastoverlap)
            elif paqcIcapF:
                gene_capped = bsaI_firstoverlap + paqcI_seqplusfour + paqcI_overhang_nterm_ctag + \
                        gene + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap + paqcI_seqplusfour + paqcI_overhang_nterm_ctag)
                capping_length_R = len(bsaI_lastoverlap)
            elif paqcIcapR:
                gene_capped = bsaI_firstoverlap + \
                        gene + paqcI_overhang_cterm_ctag + paqcI_seqplusfour.reverse_complement() + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap)
                capping_length_R = len(paqcI_overhang_cterm_ctag + paqcI_seqplusfour.reverse_complement() + \
                                   bsaI_lastoverlap)
            else:
                gene_capped = bsaI_firstoverlap + gene + bsaI_lastoverlap
                capping_length_F = len(bsaI_firstoverlap)
                capping_length_R = len(bsaI_lastoverlap)
            
            #Optimize gene if tile boundaries are not given
            if tile_boundaries is False:
                optimum_breakpoints, optimum_overlaps, optimum_scores, optimum_lengths, oligo_array_indices = \
                optimize_gene(gene_capped, 
                          max_tile_size=max_tile_size,
                          first_last_block_reduction=first_last_block_reduction,
                          block_size_range=block_size_range, 
                          slack=slack, 
                          empirical=bsaI_empirical, 
                          overhang_blacklist=overhang_blacklist)
                pprint.pprint({'Optimum Breakpoints': optimum_breakpoints, 
                       'Optimum Overlaps': optimum_overlaps, 
                       'Optimum Scores': optimum_scores})
                num_blocks[gene_name] = len(optimum_breakpoints)
            else:
                optimum_breakpoints = tile_boundaries[gene_name]
                if len(optimum_breakpoints[0])==3:
                    if len(optimum_breakpoints)>1:
                        #multiple tiles, including one at beginning of gene
                        oligo_array_indices = [[0,1]] + [[1,2]]*(len(optimum_breakpoints)-1)
                    else:
                        # one tile
                        if ((optimum_breakpoints[1]-optimum_breakpoints[0]) > (optimum_breakpoints[2]-optimum_breakpoints[1])):
                            # at end of gene
                            oligo_array_indices = [[1,2]]
                        else:
                            # at beginning of gene
                            oligo_array_indices = [[0,1]]
                else:
                    # multiple tiles, starting in the middle
                    oligo_array_indices = [[1,2]]*(len(optimum_breakpoints))
                num_blocks[gene_name] = len(optimum_breakpoints)
                
            
            #add primers for gene_F and gene_R that are repeated constantly throughout the PCRs
            #note: should probably prevalidate these primers!
            if find_pcr_primers:
                F_primer = generate_primer(gene,
                                           Fwd=True,
                                           extendtoCG=extendtoCG,
                                           smallest_primer_size=smallest_primer_size,
                                           largest_primer_size=largest_primer_size,
                                           Tm=Tm)
                F_primer = pcr_capseq + bsaI_firstoverlap + paqcI_seqplusfour + F_primer
                amp_primers[gene_name+'_gene'+'_ampF'] = F_primer
                R_primer = generate_primer(gene,
                                           Fwd=False,
                                           extendtoCG=extendtoCG,
                                           smallest_primer_size=smallest_primer_size,
                                           largest_primer_size=largest_primer_size,
                                           Tm=Tm)
                R_primer = pcr_capseq + Seq(bsaI_lastoverlap).reverse_complement() + \
                            paqcI_seqplusfour + R_primer
                amp_primers[gene_name+'_gene'+'_ampR'] = R_primer
            
            #make oligos, primers, gblocks for each block
            for i,breakpoint in enumerate(optimum_breakpoints):
                
                #find indices of breakpoint that correspond to oligo vs need to be PCRed/gblock
                pcr_indices = [[j,j+1] for j in range(len(breakpoint)-1)]
                pcr_indices.remove(oligo_array_indices[i])
                
                #find mutagenic window of oligo
                oligo_breaks = [breakpoint[j] for j in oligo_array_indices[i]]
                oligo_mutagenic_window = [int(3*np.ceil(max(oligo_breaks[0]+4-capping_length_F,3)/3)), int(3*np.floor(min(oligo_breaks[1]-capping_length_F,len(gene)-1)/3))]
                
                #subset the right block if needed
                if (all_blocks == True) | ((i+1) in blocks_to_include[r] if blocks_to_include != False else True): #subset on allowed blocks
                
                    #add pcr primers and gblocks, one segment at a time
                    for k,pcr_index in enumerate(pcr_indices):
                        piece_name = gene_name + '_block' + str(i+1) + '_s' + str(k+1)
                        pcr_breaks = [breakpoint[j] for j in pcr_index]
                                            
                        #get pcr primers
                        if find_pcr_primers:
                            if pcr_breaks[0] == breakpoint[0]: #Fragment beginning at gene start 
                                R_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=False,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                R_primer = pcr_capseq + R_primer
                                amp_primers[piece_name+'_ampR'] = R_primer
                            elif pcr_breaks[1] == breakpoint[-1]: #Fragment ending at gene end
                                F_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=True,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                F_primer = pcr_capseq + F_primer
                                amp_primers[piece_name+'_ampF'] = F_primer
                            else:
                                F_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=True,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                F_primer = pcr_capseq + F_primer
                                R_primer = generate_primer(gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)],
                                                           Fwd=False,
                                                           extendtoCG=extendtoCG,
                                                           smallest_primer_size=smallest_primer_size,
                                                           largest_primer_size=largest_primer_size,
                                                           Tm=Tm)
                                R_primer = pcr_capseq + R_primer
                                amp_primers[piece_name+'_ampF'] = F_primer
                                amp_primers[piece_name+'_ampR'] = R_primer

                        #make gblocks
                        gbl = gbl_capseq_F + gene_capped[pcr_breaks[0]:(pcr_breaks[1]+4)] + gbl_capseq_R.reverse_complement()
                        gblocks[piece_name] = gbl
                        
                    #add oligos to oligo array, checking first for validated primers if they are given
                    validated=False
                    if validated_primer_set is not False:
                        validated_combo = validated_primer_set.query('Gene == @gene_name & Block == (@i+1)')
                        if not validated_combo.empty:
                            validated=True
                            name_primer_F, primer_F, name_primer_R, primer_R = \
                                validated_combo[['Forward Name',
                                                  'Forward Primer',
                                                  'Reverse Name',
                                                  'Reverse Primer']].values[0]
                        else:
                            name_primer_F, primer_F, name_primer_R, primer_R = \
                                primer_set.iloc[oligo_primer_counter,][['Forward Name',
                                                                      'Forward Primer',
                                                                      'Reverse Name',
                                                                      'Reverse Primer']]
                            oligo_primer_counter += 1
                    else:
                        name_primer_F, primer_F, name_primer_R, primer_R = \
                                primer_set.iloc[oligo_primer_counter,][['Forward Name',
                                                                      'Forward Primer',
                                                                      'Reverse Name',
                                                                      'Reverse Primer']]
                        oligo_primer_counter += 1
                        
                    # if mutations are given, use those - otherwise make all mutations or wtonly
                    if mutations_to_use == False:
                        add_on_array = make_mutations(gene_name + '_block' + str(i+1),
                                           gene[oligo_mutagenic_window[0]:oligo_mutagenic_window[1]],
                                           region_flanks=[Seq(primer_F) + \
                                                          bsaI_seqplusone + \
                                                          gene_capped[oligo_breaks[0]:(oligo_mutagenic_window[0]+capping_length_F)] ,
                                                          gene_capped[(oligo_mutagenic_window[1]+capping_length_F):(oligo_breaks[1]+4)] + \
                                                          bsaI_seqplusone.reverse_complement() + \
                                                          Seq(primer_R).reverse_complement()],
                                           nt_start=oligo_mutagenic_window[0],
                                           wt_only=wt_only,
                                           synonymous=synonymous,
                                           stops=stops,
                                           all3ntdeletions=all3ntdeletions,
                                           codons_ranked_by_usage=codons_ranked_by_usage,
                                           aa_start=aa_start_gene)
                    else:
                        add_on_array = make_mutations(gene_name + '_block' + str(i+1),
                                           gene[oligo_mutagenic_window[0]:oligo_mutagenic_window[1]],
                                           region_flanks=[Seq(primer_F) + \
                                                          bsaI_seqplusone + \
                                                          gene_capped[oligo_breaks[0]:(oligo_mutagenic_window[0]+capping_length_F)] ,
                                                          gene_capped[(oligo_mutagenic_window[1]+capping_length_F):(oligo_breaks[1]+4)] + \
                                                          bsaI_seqplusone.reverse_complement() + \
                                                          Seq(primer_R).reverse_complement()],
                                           nt_start=oligo_mutagenic_window[0],
                                           mutation_list=mutations_to_use[(gene_name,i+1)],
                                           codons_ranked_by_usage=codons_ranked_by_usage,
                                           aa_start=aa_start_gene)
                    oligo_array.update(add_on_array)
                    amp_primer_dict.update({(gene_name,i+1): (name_primer_F,primer_F,name_primer_R,primer_R,validated)})
                    breakpoint_dict.update({(gene_name,i+1): oligo_mutagenic_window})
                    
    #Check that max oligo is less than the max oligo length
    if sum([len(s)>max_oligo_size for s in oligo_array.values()]) == 0:
        print('All oligos are below the maximum 250bp!')
    else:
        print('Some oligos are TOO BIG!')
        
    #Check for nonspecific amplification
    wt_oligos = {tuple([key.split('_')[0],
                        int((key.split('_block')[1]).split('_')[0])]
                      ):oligo_array[key] \
                     for key in oligo_array.keys() if 'WT' in key}
    nonspecific_primers = post_qc(amp_primer_dict, 
                                  wt_oligos,
                                  primer_set, 
                                  melt_temp_threshold=qc_melt_temp_threshold,
                                  check_all_primers=check_all_primers)
    
    # Check that all mutagenic windows overlap
    breakpoint_df = pd.DataFrame.from_dict(breakpoint_dict, orient='index', columns=['Mutagenesis Start','Mutagenesis End'])
    breakpoint_df.index = pd.MultiIndex.from_tuples(breakpoint_dict.keys())
    breakpoint_df = breakpoint_df.reset_index().rename(columns={'level_0':'Gene',
                                                                'level_1':'Block'})
    if (mutations_to_use == False) and (all_blocks == True):
        missed_counter = 0
        for r,gene_group_breakpoints in breakpoint_df.groupby('Gene'):
            for k,row in gene_group_breakpoints.iterrows():
                # look at whether the current row start is later than the last row end
                if row['Block'] > 1:
                    if row['Mutagenesis Start'] > end:
                        missed_counter += 1
                        print('Mutagenic window missed at ' + str(r) + ' block ' + str(row['Block']))
                start,end = row['Mutagenesis Start'],row['Mutagenesis End']
        if missed_counter == 0:
            print('All mutagenic windows overlap!')
        else:
            print(str(missed_counter) + ' number of times the mutagenic window does not close!')
                
    #Remove any oligos with additional BsaI sites or paqcI sites
    bad_oligos = []
    for name,oligo in oligo_array.items():
        #check for paqcI sites
        paqcI_F = sum([True for kmer in build_kmers(oligo, len(paqcI_seq)) if kmer==paqcI_seq])
        paqcI_R = sum([True for kmer in build_kmers(oligo.reverse_complement(), len(paqcI_seq)) if kmer==paqcI_seq])
        #check that oligo is block 1 if it contains a paqcI site in the forward orientation 
        if paqcI_F > 0:
            if ('block1' not in name) | (paqcI_F > 1):
                bad_oligos.append(name)
        #check that the oligo is block final if it contains a paqcI site in the reverse orientation
        if paqcI_R > 0:
            if ('block'+str(num_blocks[name.split('_')[0]]) not in name) | (paqcI_R > 1):
                bad_oligos.append(name)
        #check for more than one BsaI site
        bsaI_F = sum([True for kmer in build_kmers(oligo, len(bsaI_seq)) if kmer==bsaI_seq])
        bsaI_R = sum([True for kmer in build_kmers(oligo.reverse_complement(), len(bsaI_seq)) if kmer==bsaI_seq])
        if (bsaI_F != 1) | (bsaI_R != 1):
            bad_oligos.append(name)
    bad_oligos=np.unique(bad_oligos)
    for oligo_name in bad_oligos:
        del oligo_array[oligo_name]
    print(str(len(bad_oligos)) + ' oligos deleted due to errant restriction sites.')
    
    #Remove any duplicate oligos
    new_dict = {}
    seen_values = set()
    counter=0
    for key, value in oligo_array.items():
        if value not in seen_values:
            new_dict[key] = value
            seen_values.add(value)
        else:
            counter += 1
    print(str(counter) + ' oligos removed due to duplication.')
    oligo_array = new_dict
    del new_dict
    
    #write oligo array to file
    with open(oligo_file, 'w') as f:
        for key in oligo_array.keys():
            f.write("%s,%s\n"%(key,oligo_array[key]))
    f.close()
            
    #write primers to file
    if find_pcr_primers:
        primer_order_sheet = []
        for key in amp_primers.keys():
            primer_order_sheet.append(key + '\t' + \
                     str(amp_primers[key]) + \
                     '\t' + '25nm' + '\t' + 'STD\n')
        print(*primer_order_sheet)
        with open(primer_file, 'w') as f:
            for line in primer_order_sheet:
                f.write(line)
        f.close()
    
    #write amplification primer key to file
    amp_primer_key = ['Gene' + '\t' + 'Block' + '\t' + \
                      'Forward Primer Well' + '\t' + 'Forward Primer' + '\t' + \
                      'Reverse Primer Well' + '\t' + 'Reverse Primer' + '\t' + 'Validated' + '\n']
    for key in amp_primer_dict.keys():
        genename, geneblock = key[0], str(key[1])
        name_primer_F, primer_F, name_primer_R, primer_R, validated = amp_primer_dict[key]
        amp_primer_key.append(genename + '\t' + geneblock + '\t' + \
                 name_primer_F + '\t' + primer_F + '\t' + \
                 name_primer_R + '\t' + primer_R + '\t' + str(validated) + '\n')
    print(*amp_primer_key)
    with open(amp_primer_key_file, 'w') as f:
        for line in amp_primer_key:
            f.write(line)
    f.close()
    
    #write breakpoint dict to file
    breakpoint_df.to_csv(breakpoint_file, sep='\t')
    
    #write gblocks to file
    gblock_order_sheet = []
    gblock_large_order_sheet = []
    for key in gblocks.keys():
        # pad gblock if it is not 300bp for Twist
        if len(gblocks[key]) < gblock_min_size:
            gblocks[key] = Seq(randomsequencepad[0:(gblock_min_size-len(gblocks[key]))]) + gblocks[key]
        if len(gblocks[key]) < gblock_large_threshold:
            gblock_order_sheet.append(key + '\t' + \
                     str(gblocks[key]) + '\n')
        else:
            gblock_large_order_sheet.append(key + '\t' + \
                     str(gblocks[key]) + '\n')
    print(*gblock_order_sheet)
    print(*gblock_large_order_sheet)
    with open(gbl_file, 'w') as f:
        for line in gblock_order_sheet:
            f.write(line)
    f.close()
    with open(gbl_large_file, 'w') as f:
        for line in gblock_large_order_sheet:
            f.write(line)
    f.close()
    
    return oligo_array,amp_primers,gblocks,amp_primer_dict,breakpoint_df
                

In [65]:
# Make the 5' library
oligo_array,amp_primers,gblocks,amp_primer_dict,breakpoint_df = write_oligo_library({
                                            'FRED1':FRED_SB1,
                                            'VWF1':VWF_SB1},
                                          oligo_file='Johnsenlab/5p_oligos.csv',
                                          primer_file='Johnsenlab/5p_primers.tsv',
                                          gbl_file='Johnsenlab/5p_gblocks.tsv',
                                          gbl_large_file='Johnsenlab/5p_gblocks_large.tsv',
                                          amp_primer_key_file='Johnsenlab/5p_ampkey.tsv',
                                          breakpoint_file='Johnsenlab/5p_breakpoints.tsv',
                                          block_size_range=[148,190],
                                          primer_set=orthogonal_primers_touse,
                                          validated_primer_set=validated_primer_combos_filtered,
                                          aa_start={                    })


Processing gene 1: FRED1
Gene has no paqcI or BsaI sites! Performing GoldenGate optimization...
All regions are high fidelity!
{'Optimum Breakpoints': [[0, 152, 1693],
                         [0, 134, 308, 1693],
                         [0, 290, 468, 1693],
                         [0, 444, 624, 1693],
                         [0, 609, 778, 1693],
                         [0, 764, 932, 1693],
                         [0, 917, 1089, 1693],
                         [0, 1076, 1238, 1693],
                         [0, 1225, 1401, 1693],
                         [0, 1386, 1553, 1693],
                         [0, 1543, 1693]],
 'Optimum Overlaps': [['ACGA', 'TGGA', 'TTTC'],
                      ['ACGA', 'ATCT', 'CTGA', 'TTTC'],
                      ['ACGA', 'TAGG', 'TGGA', 'TTTC'],
                      ['ACGA', 'AGAA', 'TCTG', 'TTTC'],
                      ['ACGA', 'ATGT', 'AATG', 'TTTC'],
                      ['ACGA', 'AAAT', 'TGGA', 'TTTC'],
                      ['ACGA', 'ATCG', '

/opt/miniconda3/lib/python3.13/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/opt/miniconda3/lib/python3.13/site-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/opt/miniconda3/lib/python3.13/site-packages/primer3/bindings.py:305: UserWarning: Function deprecated please use "calc_heterodimer" instead
  return THERMO_ANALYSIS.calcHeterodimer(


No non-specific primers detected
All mutagenic windows overlap!
229 oligos deleted due to errant restriction sites.
911 oligos removed due to duplication.
FRED1_gene_ampF	GGCTACGGTCTCTACGACACCTGCCTAGATGCAAATAGAGCTCTCTACCTGCTTC	25nm	STD
 FRED1_gene_ampR	GGCTACGGTCTCTGAAACACCTGCCTAGAGAGTAATAGCGGGTCAGGCAC	25nm	STD
 FRED1_block1_s1_ampF	GGCTACGGTCTCTTGGACGCAAGATTTCCTCCTAGAG	25nm	STD
 FRED1_block2_s1_ampR	GGCTACGGTCTCTAGATCACTTTGCATATAGTCCCATGACAG	25nm	STD
 FRED1_block2_s2_ampF	GGCTACGGTCTCTCTGAGGTTTATGATACAGTGGTCATTACAC	25nm	STD
 FRED1_block3_s1_ampR	GGCTACGGTCTCTCCTAGCAGACCCATCCAGGG	25nm	STD
 FRED1_block3_s2_ampF	GGCTACGGTCTCTTGGAAGCCATACATATGTCTGGCAG	25nm	STD
 FRED1_block4_s1_ampR	GGCTACGGTCTCTTTCTTTCTCCCTTTGACTGGTCTGATC	25nm	STD
 FRED1_block4_s2_ampF	GGCTACGGTCTCTTCTGGCCAAGGAAAAGACACAGAC	25nm	STD
 FRED1_block5_s1_ampR	GGCTACGGTCTCTACATACTAGTAGGGCTCCAATGAGG	25nm	STD
 FRED1_block5_s2_ampF	GGCTACGGTCTCTAATGGTTATGTAAACAGATCTCTGCCAG	25nm	STD
 FRED1_block6_s1_ampR	GGCTACGGTCTCTATTTTAGGCCAGGCC

In [ ]:
# Subtract the primer pairs used in making the 5p libraries from the remaining pairs to make the list for the internal libraries

#1. Read the two primer tables
df_used = pd.read_csv(
    "Johnsenlab/5p_ampkey.tsv",
    sep="\t",
    usecols=["Forward Primer", "Reverse Primer"]
)

df_all = orthogonal_primers_touse  # keep every column

#2. Build helper columns to subracting 
df_used_norm = df_used.assign(
    FWD=df_used["Forward Primer"].str.strip().str.upper(),
    REV=df_used["Reverse Primer"].str.strip().str.upper()
)

df_all_norm = df_all.assign(
    FWD=df_all["Forward Primer"].str.strip().str.upper(),
    REV=df_all["Reverse Primer"].str.strip().str.upper()
)

#3. Identify pairs that are already used
used_index = df_used_norm.set_index(["FWD", "REV"]).index
mask       = ~df_all_norm.set_index(["FWD", "REV"]).index.isin(used_index)

#4. Keep the rows (and *all* original columns) that remain
orthogonal_primers_remaining = df_all.loc[mask].reset_index(drop=True)

print(f"{len(orthogonal_primers_remaining_iter5)} primer pairs remain.")
orthogonal_primers_remaining.head()
orthogonal_primers_remaining.to_csv("orthogonal_primers_remaining.csv", index=False)

In [ ]:
# Make make the internal libraries
oligo_array,amp_primers,gblocks,amp_primer_dict,breakpoint_df = write_oligo_library({
                                            'FRED2':FRED_SB2,
                                            'VWF2':VWF_SB2,
                                            'VWF3':VWF_SB3,
                                            'VWF4':VWF_SB4},
                                          oligo_file='Johnsenlab/int_oligos.csv',
                                          primer_file='Johnsenlab/int_primers.tsv',
                                          gbl_file='Johnsenlab/int_gblocks.tsv',
                                          gbl_large_file='Johnsenlab/int_gblocks_large.tsv',
                                          amp_primer_key_file='Johnsenlab/int_ampkey.tsv',
                                          breakpoint_file='Johnsenlab/int_breakpoints.tsv',
                                          block_size_range=[148,190],
                                          primer_set=orthogonal_primers_remaining,
                                          validated_primer_set=validated_primer_combos_filtered,
                                          aa_start={                    })


In [ ]:
# Subtract the primer pairs used in making the first two sets of libraries from the remaining pairs to make the list for the 3p libraries

#1. Read the two primer tables
df_used = pd.read_csv(
    "Johnsenlab/int_ampkey.tsv",
    sep="\t",
    usecols=["Forward Primer", "Reverse Primer"]
)

df_all = orthogonal_primers_remaining  # keep every column

#2. Build helper columns to subracting 
df_used_norm = df_used.assign(
    FWD=df_used["Forward Primer"].str.strip().str.upper(),
    REV=df_used["Reverse Primer"].str.strip().str.upper()
)

df_all_norm = df_all.assign(
    FWD=df_all["Forward Primer"].str.strip().str.upper(),
    REV=df_all["Reverse Primer"].str.strip().str.upper()
)

#3. Identify pairs that are already used
used_index = df_used_norm.set_index(["FWD", "REV"]).index
mask       = ~df_all_norm.set_index(["FWD", "REV"]).index.isin(used_index)

#4. Keep the rows (and *all* original columns) that remain
orthogonal_primers_remaining_iter2 = df_all.loc[mask].reset_index(drop=True)

print(f"{len(orthogonal_primers_remaining_iter5)} primer pairs remain.")
orthogonal_primers_remaining_iter2.head()
orthogonal_primers_remaining_iter2.to_csv("orthogonal_primers_remaining_iter2.csv", index=False)

In [ ]:
# Make the 3p libraries
oligo_array,amp_primers,gblocks,amp_primer_dict,breakpoint_df = write_oligo_library({
                                            'FRED3':FRED_SB3,
                                            'VWF5':VWF_SB5},
                                          oligo_file='Johnsenlab/3p_oligos.csv',
                                          primer_file='Johnsenlab/3p_primers.tsv',
                                          gbl_file='Johnsenlab/3p_gblocks.tsv',
                                          gbl_large_file='Johnsenlab/3p_gblocks_large.tsv',
                                          amp_primer_key_file='Johnsenlab/3p_ampkey.tsv',
                                          breakpoint_file='Johnsenlab/3p_breakpoints.tsv',
                                          block_size_range=[148,190],
                                          primer_set=orthogonal_primers_remaining_iter2,
                                          validated_primer_set=validated_primer_combos_filtered,
                                          aa_start={                    })


In [ ]:
# Okay, I have a draft of the library, now I need to see if the overhangs conflict with the superblocks
# First I need to pull the overhangs from the tiles using the breakpoint values




In [43]:
genes = {
           'FRED1':FRED_SB1,
           'FRED2':FRED_SB2,
           'FRED3':FRED_SB3,
           'VWF1':VWF_SB1,
           'VWF2':VWF_SB2,
           'VWF3':VWF_SB3,
           'VWF4':VWF_SB4,
           'VWF5':VWF_SB5
        }

